### 读取数据

In [14]:
import pandas as pd

# 1. 读取数据
df = pd.read_csv("ruc_Class25Q2_train_price.csv")
df_test = pd.read_csv("ruc_Class25Q2_test_price.csv")

C:\Users\heart\AppData\Local\Temp\ipykernel_15560\2208309609.py:4: DtypeWarning: Columns (3,32,34,43,46,49,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("ruc_Class25Q2_train_price.csv")
C:\Users\heart\AppData\Local\Temp\ipykernel_15560\2208309609.py:5: DtypeWarning: Columns (4,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv("ruc_Class25Q2_test_price.csv")


In [15]:
import pandas as pd

# 1. 定义函数：统计单个数据框的空值情况，返回无空值的列名
def get_non_null_cols(df, df_name):
    # 统计每列的空值数量
    null_counts = df.isnull().sum()
    # 筛选空值数量为0的列（无空值列）
    non_null_cols = null_counts[null_counts == 0].index.tolist()
    # 打印该数据框的空值统计
    print(f"===== {df_name} 空值统计 =====")
    print(f"总列数：{df.shape[1]}")
    print(f"无空值列数：{len(non_null_cols)}")
    print(f"{df_name} 无空值的列：{non_null_cols}\n")
    return non_null_cols

# 2. 分别获取df和df_test的无空值列
df_non_null = get_non_null_cols(df, "df")
df_test_non_null = get_non_null_cols(df_test, "df_test")

# 3. 求交集：获取两个数据框都没有空值的列
common_non_null_cols = list(set(df_non_null) & set(df_test_non_null))
# 按列名排序（可选，让结果更整齐）
common_non_null_cols.sort()

# 4. 打印最终结果
print("="*60)
print("最终结果：df 和 df_test 均无空值的列")
print(f"共 {len(common_non_null_cols)} 列：")
for i, col in enumerate(common_non_null_cols, 1):
    print(f"{i}. {col}")

===== df 空值统计 =====
总列数：55
无空值列数：13
df 无空值的列：['城市', '区域', '板块', 'Price', '所在楼层', '建筑面积', '交易时间', '交易权属', '产权所属', 'lon', 'lat', '年份', '客户反馈']

===== df_test 空值统计 =====
总列数：55
无空值列数：15
df_test 无空值的列：['ID', '城市', '区域', '板块', '所在楼层', '建筑面积', '房屋朝向', '交易时间', '交易权属', '房屋用途', '产权所属', 'lon', 'lat', '年份', '客户反馈']

最终结果：df 和 df_test 均无空值的列
共 12 列：
1. lat
2. lon
3. 交易时间
4. 交易权属
5. 产权所属
6. 区域
7. 城市
8. 客户反馈
9. 年份
10. 建筑面积
11. 所在楼层
12. 板块


### 针对八个可能的方向生成0/1变量

In [13]:
import pandas as pd

# 定义需要新增的朝向类别
directions = ['东', '南', '西', '北', '东南', '西南', '西北', '东北']
# 生成列名：朝向_东、朝向_南...
new_cols = [f'朝向_{d}' for d in directions]

# 为df新增列并赋值0
df[new_cols] = 0

# 为df_test新增列并赋值0
df_test[new_cols] = 0

### 朝向填值

In [16]:
import pandas as pd

# 定义目标方向列表
directions = ['东', '南', '西', '北', '东南', '西南', '西北', '东北']
# 对应的列名
cols = [f'朝向_{d}' for d in directions]

# 处理函数：按行设置1/0
def set_direction_flags(row):
    # 分割当前行的朝向值（处理空值和空格）
    dirs = str(row['房屋朝向']).split() if pd.notna(row['房屋朝向']) else []
    # 遍历所有方向，存在则设1
    for d, col in zip(directions, cols):
        row[col] = 1 if d in dirs else 0
    return row

# 应用函数处理df
df = df.apply(set_direction_flags, axis=1)

In [17]:
# 筛选所有以"朝向_"开头的列
direction_cols = [col for col in df.columns if col.startswith('朝向_')]

# 打印前8行的这些列
print("处理后前8行的朝向特征：")
display(df[direction_cols].head(8))

处理后前8行的朝向特征：


,朝向_东,朝向_南,朝向_西,朝向_北,朝向_东南,朝向_西南,朝向_西北,朝向_东北
0,0,1,0,1,0,0,0,0
1,0,1,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0
3,1,1,1,1,0,0,0,0
4,0,1,0,0,0,0,0,0
5,0,1,0,1,0,0,0,0
6,0,1,0,1,0,0,0,0
7,0,1,0,1,0,0,0,0


In [18]:
df_test = df_test.apply(set_direction_flags, axis=1)

#### 查看数据特性

In [4]:
summary = pd.DataFrame({
    "缺失值数量": df.isnull().sum(),
    "唯一值数量": df.nunique(),
    "数据类型": df.dtypes
})

# 4. 总行数
summary["总样本数"] = len(df)

# 5. 打印或保存
print(summary)

            缺失值数量   唯一值数量     数据类型    总样本数
城市              0      12    int64  103871
区域              0     117  float64  103871
板块              0     977  float64  103871
环线          63452      11   object  103871
Price           0  103783  float64  103871
房屋户型          580     388   object  103871
所在楼层            0     258   object  103871
建筑面积            0   15772   object  103871
套内面积        67887    9977   object  103871
房屋朝向            1     173   object  103871
建筑结构          580       7   object  103871
装修情况          580       4   object  103871
梯户比例         2619     327   object  103871
配备电梯        12351       2   object  103871
别墅类型       102428       4   object  103871
交易时间            0    1628   object  103871
交易权属            0      15   object  103871
上次交易        25449    8407   object  103871
房屋用途            1      20   object  103871
房屋年限        44510       3   object  103871
产权所属            0       2   object  103871
抵押信息       103871       0  float64  103871
房屋优势       

### 楼层只留第一个字

In [19]:
# 果担心有空白字符，可以先去除空白再取第一个字符
df['所在楼层'] = df['所在楼层'].str.strip().str[0]

In [20]:
print("所在楼层列的所有唯一取值:")
print(df_test['房屋用途'].unique())

所在楼层列的所有唯一取值:
['普通住宅' '商业办公类' '酒店式公寓' '别墅' '公寓' '公寓/公寓' '车库' '公寓（住宅）' '公寓/住宅' '商务型公寓'
 '商住两用' '新式里弄' '花园洋房' '老公寓' '商业' '商务公寓']


In [21]:
print("所有唯一取值:")
print(df['房屋朝向'].unique())

所有唯一取值:
['南 北' '东南' '东 南 西 北' '南' '南 西 北' '西' '北' '西 北' '东' '东南 西' '南 西' '东 北'
 '东 南 北' '东南 西北' '西 南' '东 西' '西南' '北 西' '东北' '南 北 西 东' '西北' '东 北 东北' '南 东'
 '东 南 西' '北 南' '西南 东' '南 北 东' '东南 北' '西 东' '南 北 西' '南 北 东 西' '北 东北'
 '东 西 北' '北 东南' '西南 东北' '东 西北' '东 南' '西南 北' '东 西 南' '南 西南' '西北 南' '东南 东北'
 '东 北 西' '北 东' '西北 东北' '北 西 东' '西 南 北' '东 东南 西 西北' '北 南 西' '南 东 西'
 '南 西南 北' '南 西北' '西南 西 南' '南 东 北' '东 东南 南' '东南 南' '西北 西 东' '北 南 东' '东南 西南'
 '西 西北' '西 北 南' '东北 西' '西南 西北' '东 西南 北' '东 东南' '西 东 北' '东南 南 北' '东北 东南'
 '东 东南 南 西南 西' '南 东北' '东南 西 北' '东南 南 东' '西 南 东 北' '东 东北' '西 东北' '西南 西'
 '南 西 东' '南 东南' '东南 西南 西北' '西 北 东' '西南 南' '西南 东南' '东北 南' '北 西南' '南 东 西南'
 '东北 西北' '东 东南 北' '西 北 东 南' '西 东 南' '西北 东' '西南 西北 东北' '东南 南 西南' '东 西南'
 '南 西南 西' '东 东南 南 北' '西 西北 北' '东 南 东南' '南 北 东北' '东 西 南 北' '南 东 东南 西南 西'
 '西 北 西北' '东北 北' '西北 北' '东 南 东北' '东南 东' '北 西北' '东北 西南' '西南 西 西北' '西 西南'
 '北 东南 东' '南 西北 西' '西北 东南' '南 西 西南 东南' '南 西南 东北' '西北 西' '东南 南 西北 北'
 '东南 南 西 西南' '北 东 东北' '东南 西南 西 西北' '西南 北 东北' '东 东南 南 西南' '南 西南 

In [74]:
print("所有唯一取值:")
print(df_test['年份'].unique())

所有唯一取值:
[2022. 2023.]


### 测试集多了自住型商品房--->商品房

In [22]:
import pandas as pd

# 1. 统计“交易权属”列为“自住型商品房”的行数
count = df_test[df_test['交易权属'] == '自住型商品房'].shape[0]
print(f"交易权属为'自住型商品房'的行数：{count}")

# 2. 将这些行的“交易权属”值改为“商品房”
df_test.loc[df_test['交易权属'] == '自住型商品房', '交易权属'] = '商品房'

交易权属为'自住型商品房'的行数：7


训练集：城市、户型、Price、面积、交易时间、lon/lat、年份、（区县、板块）---有空值

In [23]:
summary = pd.DataFrame({
    "缺失值数量": df_test.isnull().sum(),
    "唯一值数量": df_test.nunique(),
    "数据类型": df_test.dtypes
})

summary["总样本数"] = len(df_test)

print(summary)

       缺失值数量  唯一值数量     数据类型   总样本数
ID         0  34017    int64  34017
城市         0     12    int64  34017
区域         0    118  float64  34017
板块         0    954  float64  34017
环线     18340     11   object  34017
...      ...    ...      ...    ...
朝向_北       0      2    int64  34017
朝向_东南      0      2    int64  34017
朝向_西南      0      2    int64  34017
朝向_西北      0      2    int64  34017
朝向_东北      0      2    int64  34017

[63 rows x 4 columns]


In [24]:
df_test['所在楼层'] = df_test['所在楼层'].str.strip().str[0]

测试集：城市、户型、楼层、朝向、Price、面积、交易时间、lon/lat、年份、（区县、板块）---有空值

### 处理所在楼层变成虚拟变量

In [25]:
import pandas as pd

# 1. 生成所有类别的独热编码（共6个虚拟变量）
# prefix指定虚拟变量的前缀，方便区分
floor = pd.get_dummies(df['所在楼层'], prefix='楼层', drop_first=True,dtype=int)

# 3. 删除原“所在楼层”列，合并虚拟变量到原数据框
df = df.drop(columns=['所在楼层']).join(floor)

floort = pd.get_dummies(df_test['所在楼层'], prefix='楼层', drop_first=True,dtype=int)

# 删除原“所在楼层”列，合并虚拟变量到原数据框
df_test = df_test.drop(columns=['所在楼层']).join(floort)

### 交易权属变为dummies

In [26]:
# 生成虚拟变量：为每个权属类型创建0/1列（prefix指定前缀，方便识别）
dummies = pd.get_dummies(df['交易权属'], prefix='权属', drop_first=True,dtype=int)

# 删除原始“交易权属”列，合并虚拟变量到原DataFrame
df = df.drop(columns=['交易权属']).join(dummies)

# 查看结果（展示前5行）
print("转换后的数据（前5行）：")
print(df.head())

转换后的数据（前5行）：
   城市     区域      板块    环线         Price      房屋户型     建筑面积     套内面积     房屋朝向  \
0   0  109.0   150.0  二至三环  6.194049e+06  2室1厅1厨1卫    52.3㎡      NaN      南 北   
1   0   65.0   299.0  五至六环  4.354153e+06  3室1厅1厨1卫  127.44㎡   123.7㎡      南 北   
2   0   62.0   911.0  五至六环  3.321992e+06  3室2厅1厨2卫  118.02㎡  101.95㎡       东南   
3   0  123.0  1102.0   六环外  7.895656e+06  6室3厅1厨3卫  293.23㎡  293.23㎡  东 南 西 北   
4   0   81.0   295.0  三至四环  1.902960e+06     1房间1卫   39.85㎡   29.94㎡        南   

   建筑结构  ... 权属_回迁房 权属_央产房 权属_定向安置房 权属_已购公房 权属_房改房 权属_拆迁还建房 权属_私产 权属_经济适用房  \
0  混合结构  ...      0      0        0       0      0        0     0        0   
1  混合结构  ...      0      0        0       0      0        0     0        0   
2  钢混结构  ...      0      0        0       0      0        0     0        0   
3  混合结构  ...      0      0        0       0      0        0     0        0   
4  钢混结构  ...      0      0        0       0      0        0     0        0   

  权属_限价商品房  权属_集资房  
0        0

## 房屋用途

In [27]:
import pandas as pd

# ---------------- 新增：将房屋用途列的空值填充为“普通住宅” ----------------
if '房屋用途' in df.columns:
    # 统计填充前的空值数量（可选，用于验证）
    null_count_before = df['房屋用途'].isna().sum()
    if null_count_before > 0:
        print(f"检测到'房屋用途'列有 {null_count_before} 个空值，已默认填充为'普通住宅'")
    # 核心填充逻辑：空值→普通住宅，非空值保持不变
    df['房屋用途'] = df['房屋用途'].fillna('普通住宅')
else:
    print("数据集中不存在'房屋用途'列，无法进行填充和虚拟变量转换")
    # 若列不存在，直接退出后续流程
    exit()

# ---------------- 原有虚拟变量生成逻辑（不变） ----------------
# 生成虚拟变量：为每个权属类型创建0/1列（prefix指定前缀，方便识别）
dummf = pd.get_dummies(df['房屋用途'], prefix='用途', drop_first=True, dtype=int)

# 删除原始“房屋用途”列，合并虚拟变量到原DataFrame
df = df.drop(columns=['房屋用途']).join(dummf)

# 查看结果（展示前5行）
print("\n转换后的数据（前5行）：")
print(df.head())

检测到'房屋用途'列有 1 个空值，已默认填充为'普通住宅'

转换后的数据（前5行）：
   城市     区域      板块    环线         Price      房屋户型     建筑面积     套内面积     房屋朝向  \
0   0  109.0   150.0  二至三环  6.194049e+06  2室1厅1厨1卫    52.3㎡      NaN      南 北   
1   0   65.0   299.0  五至六环  4.354153e+06  3室1厅1厨1卫  127.44㎡   123.7㎡      南 北   
2   0   62.0   911.0  五至六环  3.321992e+06  3室2厅1厨2卫  118.02㎡  101.95㎡       东南   
3   0  123.0  1102.0   六环外  7.895656e+06  6室3厅1厨3卫  293.23㎡  293.23㎡  东 南 西 北   
4   0   81.0   295.0  三至四环  1.902960e+06     1房间1卫   39.85㎡   29.94㎡        南   

   建筑结构  ... 用途_商务公寓 用途_商务型公寓 用途_四合院 用途_底商 用途_新式里弄 用途_普通住宅 用途_老公寓 用途_花园洋房  \
0  混合结构  ...       0        0      0     0       0       1      0       0   
1  混合结构  ...       0        0      0     0       0       1      0       0   
2  钢混结构  ...       0        0      0     0       0       1      0       0   
3  混合结构  ...       0        0      0     0       0       0      0       0   
4  钢混结构  ...       0        0      0     0       0       0      0       0   

   用途

In [28]:
# 生成虚拟变量：为每个权属类型创建0/1列（prefix指定前缀，方便识别）
dummft = pd.get_dummies(df_test['房屋用途'], prefix='用途', drop_first=True,dtype=int)

df_test = df_test.drop(columns=['房屋用途']).join(dummft)

# 查看结果（展示前5行）
print("转换后的数据（前5行）：")
print(df_test.head())

转换后的数据（前5行）：
        ID  城市     区域      板块    环线      房屋户型     建筑面积     套内面积 房屋朝向  建筑结构  \
0  1000000   0  109.0   367.0  二至三环  3室2厅1厨2卫  282.02㎡      NaN  南 北  钢混结构   
1  1000001   0   28.0   606.0  五至六环  2室1厅1厨1卫   88.42㎡   71.78㎡  南 北  钢混结构   
2  1000002   0  123.0  1110.0  五至六环  3室1厅1厨2卫  175.52㎡  139.86㎡   西北  混合结构   
3  1000003   0   65.0   555.0   六环外  2室1厅1厨1卫  106.13㎡      NaN  南 北  钢混结构   
4  1000004   0  109.0   990.0   二环内  3室2厅1厨2卫   116.8㎡      NaN  南 北  混合结构   

   ... 用途_商业办公类 用途_商住两用 用途_商务公寓 用途_商务型公寓 用途_新式里弄 用途_普通住宅 用途_老公寓 用途_花园洋房 用途_车库  \
0  ...        0       0       0        0       0       1      0       0     0   
1  ...        0       0       0        0       0       1      0       0     0   
2  ...        0       0       0        0       0       1      0       0     0   
3  ...        0       0       0        0       0       1      0       0     0   
4  ...        0       0       0        0       0       1      0       0     0   

   用途_酒店式公寓  
0         0  
1  

In [29]:
# 生成虚拟变量：为每个权属类型创建0/1列（prefix指定前缀，方便识别）
dummiest = pd.get_dummies(df_test['交易权属'], prefix='权属', drop_first=True,dtype=int)

# 删除原始“交易权属”列，合并虚拟变量到原DataFrame
df_test = df_test.drop(columns=['交易权属']).join(dummiest)

# 查看结果（展示前5行）
print("转换后的数据（前5行）：")
print(df_test.head())

转换后的数据（前5行）：
        ID  城市     区域      板块    环线      房屋户型     建筑面积     套内面积 房屋朝向  建筑结构  \
0  1000000   0  109.0   367.0  二至三环  3室2厅1厨2卫  282.02㎡      NaN  南 北  钢混结构   
1  1000001   0   28.0   606.0  五至六环  2室1厅1厨1卫   88.42㎡   71.78㎡  南 北  钢混结构   
2  1000002   0  123.0  1110.0  五至六环  3室1厅1厨2卫  175.52㎡  139.86㎡   西北  混合结构   
3  1000003   0   65.0   555.0   六环外  2室1厅1厨1卫  106.13㎡      NaN  南 北  钢混结构   
4  1000004   0  109.0   990.0   二环内  3室2厅1厨2卫   116.8㎡      NaN  南 北  混合结构   

   ... 权属_商品房 权属_央产房 权属_定向安置房 权属_已购公房 权属_房改房 权属_拆迁还建房 权属_私产 权属_经济适用房  \
0  ...      0      0        0       0      0        0     1        0   
1  ...      1      0        0       0      0        0     0        0   
2  ...      1      0        0       0      0        0     0        0   
3  ...      1      0        0       0      0        0     0        0   
4  ...      0      0        0       1      0        0     0        0   

   权属_限价商品房 权属_集资房  
0         0      0  
1         0      0  
2         0      0  
3

## 产权所属

In [30]:
import numpy as np
DECORATION_COL = "产权所属"  # 原始装修列名
DUMMY_COL = "is_共有"    # 生成的虚拟变量列名
TARGET_VALUE = "共有"   # 目标值（匹配此值则虚拟变量为1）

print("=" * 50)
print("处理 df：")

df[DUMMY_COL] = np.where(
    df[DECORATION_COL] == TARGET_VALUE,  
    1,  # 赋值1
    0   # 不满足则赋值0（包括空值和其他装修类型）
)

# 验证结果：统计各情况的行数
print(f"df 中 {TARGET_VALUE} 行数：{len(df[df[DECORATION_COL] == TARGET_VALUE])} → {DUMMY_COL}=1")
print(f"df 中 {DECORATION_COL} 为空的行数：{len(df[df[DECORATION_COL].isna()])} → {DUMMY_COL}=0")
print(f"df 中其他类型行数：{len(df[(df[DECORATION_COL] != TARGET_VALUE) & (df[DECORATION_COL].notna())])} → {DUMMY_COL}=0")

# 可选：打印前5行预览（显示原始装修列和虚拟变量列）
print("\ndf 前5行预览（原始产权列 + 虚拟变量列）：")
print(df[[DECORATION_COL, DUMMY_COL]].head().to_string(index=False))


# ---------------- 3. 为 df_test 生成"是否精装修"虚拟变量（逻辑与df一致） ----------------
print("\n" + "=" * 50)
print("处理 df_test：")
df_test[DUMMY_COL] = np.where(
    df_test[DECORATION_COL] == TARGET_VALUE,
    1,
    0
)

# 验证结果
print(f"df_test 中 {TARGET_VALUE} 行数：{len(df_test[df_test[DECORATION_COL] == TARGET_VALUE])} → {DUMMY_COL}=1")
print(f"df_test 中 {DECORATION_COL} 为空的行数：{len(df_test[df_test[DECORATION_COL].isna()])} → {DUMMY_COL}=0")
print(f"df_test 中其他装修类型行数：{len(df_test[(df_test[DECORATION_COL] != TARGET_VALUE) & (df_test[DECORATION_COL].notna())])} → {DUMMY_COL}=0")

# 可选：打印前5行预览
print("\ndf_test 前5行预览（原始装修列 + 虚拟变量列）：")
print(df_test[[DECORATION_COL, DUMMY_COL]].head().to_string(index=False))

# 最终确认
print("\n" + "=" * 50)
print("最终确认：")
print(f"df 的列名: {df.columns.tolist()}")
print(f"df_test 的列名: {df_test.columns.tolist()}")
print(f"df 中 '{DUMMY_COL}' 列已创建，值分布:")
print(df[DUMMY_COL].value_counts())

处理 df：
df 中 共有 行数：30147 → is_共有=1
df 中 产权所属 为空的行数：0 → is_共有=0
df 中其他类型行数：73724 → is_共有=0

df 前5行预览（原始产权列 + 虚拟变量列）：
产权所属  is_共有
 非共有      0
 非共有      0
 非共有      0
 非共有      0
 非共有      0

处理 df_test：
df_test 中 共有 行数：9816 → is_共有=1
df_test 中 产权所属 为空的行数：0 → is_共有=0
df_test 中其他装修类型行数：24201 → is_共有=0

df_test 前5行预览（原始装修列 + 虚拟变量列）：
产权所属  is_共有
 非共有      0
 非共有      0
 非共有      0
 非共有      0
 非共有      0

最终确认：
df 的列名: ['城市', '区域', '板块', '环线', 'Price', '房屋户型', '建筑面积', '套内面积', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '上次交易', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_西南', '朝向_西北', '朝向_东北', '楼层_低', '楼层_地', '楼层_底', '楼层_顶', '楼层_高', '权属_二类经济适用房', '权属_动迁安置房', '权属_售后公房', '权属_商品房', '权属_回迁房', '权属_央产

### 别墅类型

In [31]:
import pandas as pd

# 定义目标别墅类型（仅为这4类生成虚拟变量，空值视为非别墅）
target_villa_types = ['叠拼', '独栋', '联排', '双拼']

# ---------------- 1. 处理df的“别墅类型”列 ----------------
print("===== 处理 df 的 '别墅类型' 列 =====")
if '别墅类型' in df.columns:
    # 步骤1：先将空值填充为统一标识（如'非别墅'），避免空值干扰
    df['别墅类型_temp'] = df['别墅类型'].fillna('非别墅')
    
    # 步骤2：仅为目标别墅类型生成虚拟变量（prefix='别墅'，确保列名清晰）
    df_villa_dummies = pd.get_dummies(
        df['别墅类型_temp'],
        prefix='别墅',
        columns=['别墅类型_temp'],
        dtype=int
    )
    
    # 步骤3：筛选出目标4类的虚拟变量列（排除'非别墅'对应的列）
    df_villa_dummies = df_villa_dummies[[col for col in df_villa_dummies.columns 
                                       if any(villa in col for villa in target_villa_types)]]
    
    # 步骤4：合并虚拟变量到原数据集，删除临时列
    df = pd.concat([df, df_villa_dummies], axis=1).drop(columns=['别墅类型_temp'])
    
    print(f"df 生成的别墅类型虚拟变量：{df_villa_dummies.columns.tolist()}")
    print(f"df 虚拟变量列数：{len(df_villa_dummies.columns)}（预期4列）\n")
else:
    print("df 中不存在 '别墅类型' 列，跳过处理\n")

# ---------------- 2. 处理df_test的“别墅类型”列 ----------------
print("===== 处理 df_test 的 '别墅类型' 列 =====")
if '别墅类型' in df_test.columns:
    # 同df处理逻辑：空值→'非别墅'，仅生成目标4类虚拟变量
    df_test['别墅类型_temp'] = df_test['别墅类型'].fillna('非别墅')
    
    df_test_villa_dummies = pd.get_dummies(
        df_test['别墅类型_temp'],
        prefix='别墅',
        columns=['别墅类型_temp'],
        dtype=int
    )
    
    df_test_villa_dummies = df_test_villa_dummies[[col for col in df_test_villa_dummies.columns 
                                                  if any(villa in col for villa in target_villa_types)]]
    
    df_test = pd.concat([df_test, df_test_villa_dummies], axis=1).drop(columns=['别墅类型_temp'])
    
    print(f"df_test 生成的别墅类型虚拟变量：{df_test_villa_dummies.columns.tolist()}")
    print(f"df_test 虚拟变量列数：{len(df_test_villa_dummies.columns)}（预期4列）")
else:
    print("df_test 中不存在 '别墅类型' 列，跳过处理")

===== 处理 df 的 '别墅类型' 列 =====
df 生成的别墅类型虚拟变量：['别墅_双拼', '别墅_叠拼', '别墅_独栋', '别墅_联排']
df 虚拟变量列数：4（预期4列）

===== 处理 df_test 的 '别墅类型' 列 =====
df_test 生成的别墅类型虚拟变量：['别墅_双拼', '别墅_叠拼', '别墅_独栋', '别墅_联排']
df_test 虚拟变量列数：4（预期4列）


#### 正则提取面积

In [32]:
import numpy as np
import re

# 提取数字的函数
def extract_numeric(value):
    if pd.isnull(value):
        return np.nan
    # 提取所有数字（包括小数）
    nums = re.findall(r"\d+\.?\d*", str(value))
    if not nums:
        return np.nan
    nums = list(map(float, nums))
    # 若为区间（如2.61-2.63），取平均值
    return np.mean(nums)

# 需要处理的列
cols_to_clean = ["建筑面积", "燃气费", "供热费"]

# 检查哪些列在 DataFrame 中存在
cols_to_clean = [col for col in cols_to_clean if col in df.columns]

# 对每列应用提取函数
for col in cols_to_clean:
    df[col] = df[col].apply(extract_numeric)

# 查看结果前几行
print(df[cols_to_clean].head())


     建筑面积   燃气费   供热费
0   52.30  2.61  30.0
1  127.44  2.61   NaN
2  118.02  2.61  30.0
3  293.23  2.62   NaN
4   39.85  2.62  37.5


In [35]:
# 提取数字的函数
def extract_numeric(value):
    if pd.isnull(value):
        return np.nan
    # 提取所有数字（包括小数）
    nums = re.findall(r"\d+\.?\d*", str(value))
    if not nums:
        return np.nan
    nums = list(map(float, nums))
    # 若为区间（如2.61-2.63），取平均值
    return np.mean(nums)

# 需要处理的列
cols_to_clean = ["建筑面积","燃气费", "供热费"]

# 检查哪些列在 DataFrame 中存在
cols_to_clean = [col for col in cols_to_clean if col in df_test.columns]

# 对每列应用提取函数
for col in cols_to_clean:
    df_test[col] = df_test[col].apply(extract_numeric)

# 查看结果前几行
print(df_test[cols_to_clean].head())


     建筑面积   燃气费   供热费
0  282.02  2.61   NaN
1   88.42  2.61  30.0
2  175.52  2.61  30.0
3  106.13  2.61  30.0
4  116.80  2.62  27.0


#### 准备X矩阵训练集

In [36]:
import pandas as pd

# 分类变量列名（需要转为虚拟变量）
categorical_cols = ["城市", "区域", "板块"]
# 连续变量列名（直接保留原始值）
continuous_cols = ["建筑面积","lon","lat"]
binary_cols = ["is_共有"] 
# 1. 提取已有的虚拟变量（楼层、权属开头的列）
existing_dummies = df.filter(regex='^楼层_|^权属_|^用途_|^别墅_|^朝向_')

# 2. 分别对每个分类列创建虚拟变量
cat_dummies_list = []
for col in categorical_cols:
    col_dummies = pd.get_dummies(df[col], prefix=col, drop_first=True,dtype=int)
    cat_dummies_list.append(col_dummies)

# 合并所有分类变量的虚拟变量
cat_dummies = pd.concat(cat_dummies_list, axis=1)

# 3. 提取连续变量
continuous_vars = df[continuous_cols].copy()

binary_vars = df[binary_cols].copy()  # 提取is_共有列

# 4. 合并所有部分，组成训练矩阵X
X = pd.concat(
    [existing_dummies, cat_dummies, continuous_vars, binary_vars], 
    axis=1
)

# 查看结果
print("X的列名：")
print(X.columns.tolist())
print(f"\nX的形状：{X.shape}")
print("\n各类变量数量：")
print(f"- 已有虚拟变量：{existing_dummies.shape[1]}列")
print(f"- 新分类虚拟变量：{cat_dummies.shape[1]}列") 
print(f"- 连续变量：{continuous_vars.shape[1]}列")
print(f"- 总计：{X.shape[1]}列")

X的列名：
['朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_西南', '朝向_西北', '朝向_东北', '楼层_低', '楼层_地', '楼层_底', '楼层_顶', '楼层_高', '权属_二类经济适用房', '权属_动迁安置房', '权属_售后公房', '权属_商品房', '权属_回迁房', '权属_央产房', '权属_定向安置房', '权属_已购公房', '权属_房改房', '权属_拆迁还建房', '权属_私产', '权属_经济适用房', '权属_限价商品房', '权属_集资房', '用途_公寓', '用途_公寓/住宅', '用途_公寓/公寓', '用途_公寓（住宅）', '用途_写字楼', '用途_别墅', '用途_商业', '用途_商业办公类', '用途_商住两用', '用途_商务公寓', '用途_商务型公寓', '用途_四合院', '用途_底商', '用途_新式里弄', '用途_普通住宅', '用途_老公寓', '用途_花园洋房', '用途_车库', '用途_酒店式公寓', '别墅_双拼', '别墅_叠拼', '别墅_独栋', '别墅_联排', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '城市_7', '城市_8', '城市_9', '城市_10', '城市_11', '区域_4.0', '区域_5.0', '区域_7.0', '区域_8.0', '区域_9.0', '区域_10.0', '区域_11.0', '区域_12.0', '区域_13.0', '区域_14.0', '区域_15.0', '区域_16.0', '区域_17.0', '区域_18.0', '区域_19.0', '区域_20.0', '区域_21.0', '区域_22.0', '区域_23.0', '区域_24.0', '区域_26.0', '区域_27.0', '区域_28.0', '区域_29.0', '区域_30.0', '区域_31.0', '区域_32.0', '区域_33.0', '区域_34.0', '区域_35.0', '区域_36.0', '区域_37.0', '区域_38.0', '区域_39.0', '区域_41.0', '区域_42.0', '区域_43.0',

In [37]:
df['log_Price'] = np.log(df['Price'])  # 自然对数（ln）

# 4. 提取因变量y
y = df['log_Price']

# 查看结果
print("对数转换后的因变量前5行：")
print(y.head())

对数转换后的因变量前5行：
0    15.639100
1    15.286641
2    15.016075
3    15.881823
4    14.458921
Name: log_Price, dtype: float64


In [38]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
# ---------------- 2. 拟合OLS模型（训练集） ----------------
# 添加常数项（OLS模型必须包含截距项）
X_train= sm.add_constant(X)

# 拟合OLS回归：logPrice ~ 常数 + 所有自变量
model = sm.OLS(y, X_train).fit()

In [39]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_Price   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.917
Method:                 Least Squares   F-statistic:                     1078.
Date:                Wed, 29 Oct 2025   Prob (F-statistic):               0.00
Time:                        23:12:04   Log-Likelihood:                 1583.0
No. Observations:              103871   AIC:                            -1040.
Df Residuals:                  102808   BIC:                             9113.
Df Model:                        1062                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         70.5540      6.797     10.380      0.0

### 训练集变量生成

In [40]:
import pandas as pd

# 分类变量列名（需要转为虚拟变量）
categorical_cols = ["城市", "区域", "板块"]
# 连续变量列名（直接保留原始值）
continuous_cols = ["建筑面积","lon","lat"]
binary_cols = ["is_共有"]  # 0/1变量列
# 1. 提取已有的虚拟变量（楼层、权属开头的列）
existing_dummies = df_test.filter(regex='^楼层_|^权属_|^用途|^_别墅|^朝向_')

# 2. 分别对每个分类列创建虚拟变量
cat_dummies_list = []
for col in categorical_cols:
    col_dummies = pd.get_dummies(df_test[col], prefix=col, drop_first=True,dtype=int)
    cat_dummies_list.append(col_dummies)

# 合并所有分类变量的虚拟变量
cat_dummies = pd.concat(cat_dummies_list, axis=1)

# 3. 提取连续变量
continuous_vars = df_test[continuous_cols].copy()
binary_vars = df_test[binary_cols].copy()  # 提取is_共有列
# 4. 合并所有部分，组成矩阵X_test
X_test = pd.concat(
    [df_test['ID'],existing_dummies, cat_dummies, continuous_vars, binary_vars], 
    axis=1
)

# 查看结果
print("X_test的列名：")
print(X_test.columns.tolist())
print(f"\nX_test的形状：{X_test.shape}")
print("\n各类变量数量：")
print(f"- 已有虚拟变量：{existing_dummies.shape[1]}列")
print(f"- 新分类虚拟变量：{cat_dummies.shape[1]}列") 
print(f"- 连续变量：{continuous_vars.shape[1]}列")
print(f"- 总计：{X_test.shape[1]}列")

X_test的列名：
['ID', '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_西南', '朝向_西北', '朝向_东北', '楼层_低', '楼层_地', '楼层_底', '楼层_顶', '楼层_高', '用途_公寓/住宅', '用途_公寓/公寓', '用途_公寓（住宅）', '用途_别墅', '用途_商业', '用途_商业办公类', '用途_商住两用', '用途_商务公寓', '用途_商务型公寓', '用途_新式里弄', '用途_普通住宅', '用途_老公寓', '用途_花园洋房', '用途_车库', '用途_酒店式公寓', '权属_二类经济适用房', '权属_动迁安置房', '权属_售后公房', '权属_商品房', '权属_央产房', '权属_定向安置房', '权属_已购公房', '权属_房改房', '权属_拆迁还建房', '权属_私产', '权属_经济适用房', '权属_限价商品房', '权属_集资房', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '城市_7', '城市_8', '城市_9', '城市_10', '城市_11', '区域_3.0', '区域_4.0', '区域_5.0', '区域_7.0', '区域_8.0', '区域_9.0', '区域_10.0', '区域_11.0', '区域_12.0', '区域_13.0', '区域_14.0', '区域_15.0', '区域_16.0', '区域_17.0', '区域_18.0', '区域_19.0', '区域_20.0', '区域_21.0', '区域_22.0', '区域_23.0', '区域_24.0', '区域_26.0', '区域_27.0', '区域_28.0', '区域_29.0', '区域_30.0', '区域_32.0', '区域_33.0', '区域_34.0', '区域_35.0', '区域_36.0', '区域_37.0', '区域_38.0', '区域_39.0', '区域_41.0', '区域_42.0', '区域_43.0', '区域_44.0', '区域_45.0', '区域_46.0', '区域_47.0', '区域_49.0', '区域_50.0', '区域_51.

In [41]:
# 检查X_test是否有空值（0不算空）
null_count = X_test.isnull().sum().sum()

if null_count == 0:
    print("✓ X_test没有空值")
else:
    print(f"✗ X_test有{null_count}个空值")
    # 显示具体哪些列有空值
    null_cols = X_test.isnull().sum()
    null_cols = null_cols[null_cols > 0]
    print("有空值的列:")
    for col, count in null_cols.items():
        print(f"  - {col}: {count}个空值")

✓ X_test没有空值


In [192]:
print(X_train.shape)

(103871, 1150)


In [42]:
import pandas as pd

def check_nulls(X_test):
    """检查X_test中的空值，返回是否有缺失值及缺失值统计"""
    
    # 计算每列缺失值数量
    null_counts = X_test.isnull().sum()
    # 筛选有缺失值的列
    null_cols = null_counts[null_counts > 0].to_dict()
    # 判断是否存在缺失值
    has_nulls = len(null_cols) > 0
    
    return has_nulls, null_cols

# 使用示例
has_null, null_stats = check_nulls(X_test)
print("是否存在空值：", has_null)
print("空值统计（列名: 缺失数量）：", null_stats)

是否存在空值： False
空值统计（列名: 缺失数量）： {}


### 测试集中新区域、板块处理

In [43]:
import pandas as pd

def process_test(X_train, X_test, id_col='ID'):  # 假设ID列名为'ID'，可根据实际修改
    # 确保输入是DataFrame
    if not isinstance(X_train, pd.DataFrame):
        X_train = pd.DataFrame(X_train)
    if not isinstance(X_test, pd.DataFrame):
        X_test = pd.DataFrame(X_test)
    
    train_cols = list(X_train.columns)
    train_set = set(train_cols)
    test_set = set(X_test.columns)
    
    # 1. 计算特殊列（排除ID列）
    extra_cols = test_set - train_set  # 测试集有而训练集没有的列
    special_cols = [col for col in extra_cols if col != id_col]  # ID不视为特殊列
    
    # 2. 添加训练集有而测试集没有的列（值为0）
    missing_cols = train_set - test_set
    for col in missing_cols:
        X_test[col] = 0
    
    # 3. 临时保留所有必要列（训练集列 + ID列（若存在） + 其他特殊列）
    has_id = id_col in X_test.columns  # 检查测试集是否有ID列
    # 构建包含ID和特殊列的临时列列表
    all_cols = train_cols.copy()
    if has_id:
        all_cols.append(id_col)  # 加入ID列
    all_cols.extend(special_cols)  # 加入其他特殊列
    X_test_temp = X_test.reindex(columns=all_cols, fill_value=0)  # 确保列存在
    
    # 4. 提取特殊行（基于非ID的特殊列中是否有1）
    if special_cols:
        has_one = X_test_temp[special_cols].eq(1).any(axis=1)
    else:
        has_one = pd.Series(False, index=X_test_temp.index)
    special_rows_df = X_test_temp.loc[has_one].copy()
    
    # 5. 最终清洗后的测试集：保留训练集列 + ID列（若存在），删除其他特殊列
    cleaned_cols = train_cols.copy()
    if has_id and id_col not in train_cols:  # 若ID不在训练集列中，额外保留
        cleaned_cols.append(id_col)
    X_test_cleaned = X_test_temp[cleaned_cols].copy()
    
    return X_test_cleaned, special_rows_df, special_cols

In [44]:
import pandas as pd

required_cols = ['权属_自住型商品房', '权属_商品房']
missing_cols = [col for col in required_cols if col not in X_test.columns]

if missing_cols:
    print(f"警告：X_test中缺少必要的列，无法完成操作。缺失列：{missing_cols}")
else:
    # 1. 遍历并修改值：当“权属_自住型商品房”为1时，同步更新两列
    # 直接通过布尔索引批量修改（比循环更高效）
    mask = X_test['权属_自住型商品房'] == 1  # 筛选目标行
    X_test.loc[mask, '权属_商品房'] += 1    # 目标行的“权属_型商品房”加1
    X_test.loc[mask, '权属_自住型商品房'] = 0  # 目标行的“权属_自住型商品房”改为0
    
    # 2. 删除“权属_自住型商品房”列
    X_test = X_test.drop(columns=['权属_自住型商品房'])
    
    # 3. 验证结果（可选）
    print("操作完成！验证修改结果：")
    # 查看修改后的“权属_型商品房”列（原“权属_自住型商品房”为1的行）
    print("原'权属_自住型商品房'为1的行，修改后'权属_商品房'的值：")
    print(X_test.loc[mask, '权属_型商品房'].head())  # 应比原来多1
    print("\n当前X_test的列名：", X_test.columns.tolist())  # 已无“权属_自住型商品房”列

警告：X_test中缺少必要的列，无法完成操作。缺失列：['权属_自住型商品房']


In [45]:
# 已有 X_train（训练集特征矩阵）与 X_test（测试集 get_dummies 后或部分变量矩阵）
X_test_cleaned, special_rows_df, special_cols = process_test(X_train, X_test)

print("cleaned shape:", X_test_cleaned.shape)
print("special rows shape:", special_rows_df.shape)
# 查看前几行
display(X_test_cleaned.head())
display(special_rows_df.head())


cleaned shape: (34017, 1159)
special rows shape: (228, 1185)


,const,朝向_东,朝向_南,朝向_西,朝向_北,朝向_东南,朝向_西南,朝向_西北,朝向_东北,楼层_低,...,板块_1181.0,板块_1182.0,板块_1183.0,板块_1184.0,板块_1186.0,建筑面积,lon,lat,is_共有,ID
0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,282.02,117.389491,40.901030,0,1000000
1,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,88.42,117.376625,40.767478,0,1000001
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,175.52,117.631276,41.063635,0,1000002
3,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,106.13,117.186216,41.163738,0,1000003
4,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,116.80,117.400114,40.959679,0,1000004


,const,朝向_东,朝向_南,朝向_西,朝向_北,朝向_东南,朝向_西南,朝向_西北,朝向_东北,楼层_低,...,板块_726.0,板块_144.0,板块_1156.0,板块_634.0,板块_1127.0,板块_499.0,板块_110.0,区域_3.0,板块_1056.0,板块_559.0
7047,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9491,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10803,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11661,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11681,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [46]:
print(special_cols)

['板块_859.0', '板块_848.0', '板块_1112.0', '板块_659.0', '板块_699.0', '板块_278.0', '板块_507.0', '板块_613.0', '区域_61.0', '板块_706.0', '板块_10.0', '区域_79.0', '板块_77.0', '板块_691.0', '板块_1099.0', '板块_358.0', '板块_726.0', '板块_144.0', '板块_1156.0', '板块_634.0', '板块_1127.0', '板块_499.0', '板块_110.0', '区域_3.0', '板块_1056.0', '板块_559.0']


In [43]:
def print_special_rows_values(special_rows_df, special_cols, id_col='ID', verbose=True):
    """
    打印 special_rows_df 中每个特殊元组（每行）在 special_cols（特殊列）上的具体值
    """
    # ---------------- 1. 数据校验：避免空数据或无特殊列 ----------------
    if special_rows_df.empty:
        print("⚠️ special_rows_df 为空，无特殊元组可打印")
        return
    if not special_cols:
        print("⚠️ special_cols 为空，无特殊列可打印")
        return

    # ---------------- 2. 遍历每个特殊元组（每行），打印特殊列值 ----------------
    print("=" * 80)
    print(f"【special_rows_df 特殊列值详情】共 {len(special_rows_df)} 个特殊元组，{len(special_cols)} 个特殊列")
    print("=" * 80)

    for row_idx, (tuple_idx, special_tuple) in enumerate(special_rows_df.iterrows(), 1):
        # 打印元组基本信息（行号、原始索引、ID（若有））
        id_value = special_tuple[id_col] if id_col in special_tuple.index else "无"
        print(f"\n{row_idx}. 特殊元组 - 原始索引：{tuple_idx} | ID：{id_value}")
        print("-" * 50)

        # 遍历每个特殊列，提取当前元组的对应值
        for col in special_cols:
            # 处理“列不在当前元组中”的情况（视为值为0或缺失）
            if col not in special_tuple.index:
                col_value = "缺失（视为0）"
            else:
                # 提取值并统一格式（避免数值/字符串混杂）
                col_value = special_tuple[col]
                # 尝试转为数值型，方便查看（如字符串"1"转1）
                try:
                    col_value = pd.to_numeric(col_value, errors='coerce')
                    # 数值型缺失值标注
                    if pd.isna(col_value):
                        col_value = "NaN（视为0）"
                except:
                    # 非数值型（如字符串"未知"）直接保留
                    pass

            # 打印特殊列名和对应值（格式：列名 → 值）
            print(f"  {col:15} → {col_value}")

    # 补充说明：特殊列值的含义（虚拟变量通常0/1，1代表“属于该类别”）
    print("\n" + "=" * 80)
    print("【说明】")
    print("- 特殊列多为虚拟变量（0/1编码），1代表当前元组属于该类别，0代表不属于")
    print("- 标注“缺失（视为0）”“NaN（视为0）”的列，按虚拟变量逻辑默认属于“不属于该类别”")
    print("=" * 80)


# ---------------- 使用示例（衔接你的代码流程） ----------------
# 假设你已有以下变量（从之前的 align_test_to_train 函数获取）：
# special_rows_df：特殊元组数据框
# special_cols：测试集特殊列列表

# 调用函数打印特殊列值
print_special_rows_values(
    special_rows_df=special_rows_df,
    special_cols=special_cols,
    id_col='ID'  # 若你的ID列名不是'ID'，修改为实际列名（如'id'/'编号'）
)

【special_rows_df 特殊列值详情】共 228 个特殊元组，26 个特殊列

1. 特殊元组 - 原始索引：7047 | ID：1007047.0
--------------------------------------------------
  板块_499.0        → 0.0
  板块_278.0        → 0.0
  板块_659.0        → 0.0
  区域_3.0          → 0.0
  板块_706.0        → 0.0
  板块_559.0        → 0.0
  板块_1056.0       → 0.0
  区域_79.0         → 0.0
  板块_77.0         → 0.0
  板块_613.0        → 0.0
  板块_507.0        → 0.0
  板块_848.0        → 0.0
  板块_358.0        → 0.0
  板块_110.0        → 0.0
  板块_699.0        → 0.0
  板块_634.0        → 0.0
  区域_61.0         → 0.0
  板块_144.0        → 1.0
  板块_691.0        → 0.0
  板块_726.0        → 0.0
  板块_1099.0       → 0.0
  板块_10.0         → 0.0
  板块_1156.0       → 0.0
  板块_1127.0       → 0.0
  板块_1112.0       → 0.0
  板块_859.0        → 0.0

2. 特殊元组 - 原始索引：9491 | ID：1009491.0
--------------------------------------------------
  板块_499.0        → 0.0
  板块_278.0        → 0.0
  板块_659.0        → 0.0
  区域_3.0          → 0.0
  板块_706.0        → 0.0
  板块_559.0        → 0.0
  板块_1056.0   

In [44]:
for idx, row in special_rows_df.iterrows():
    tuple_id = row['ID']
    matching_row = df_test[df_test['ID'] == tuple_id]
    
    if not matching_row.empty:
        city_val = matching_row['城市'].iloc[0]
        region_val = matching_row['区域'].iloc[0]
        
        if city_val == 0 or region_val == 1:
            print(f"ID: {tuple_id}, 城市: {city_val}, 区域: {region_val}")

ID: 1007047.0, 城市: 0, 区域: 95.0
ID: 1014375.0, 城市: 2, 区域: 1.0
ID: 1014376.0, 城市: 2, 区域: 1.0
ID: 1014377.0, 城市: 2, 区域: 1.0
ID: 1014378.0, 城市: 2, 区域: 1.0
ID: 1014379.0, 城市: 2, 区域: 1.0
ID: 1014381.0, 城市: 2, 区域: 1.0
ID: 1014382.0, 城市: 2, 区域: 1.0
ID: 1014383.0, 城市: 2, 区域: 1.0
ID: 1014384.0, 城市: 2, 区域: 1.0
ID: 1014385.0, 城市: 2, 区域: 1.0
ID: 1014386.0, 城市: 2, 区域: 1.0
ID: 1014387.0, 城市: 2, 区域: 1.0
ID: 1014388.0, 城市: 2, 区域: 1.0
ID: 1014389.0, 城市: 2, 区域: 1.0
ID: 1014390.0, 城市: 2, 区域: 1.0
ID: 1014391.0, 城市: 2, 区域: 1.0
ID: 1014392.0, 城市: 2, 区域: 1.0
ID: 1014393.0, 城市: 2, 区域: 1.0
ID: 1014394.0, 城市: 2, 区域: 1.0
ID: 1014395.0, 城市: 2, 区域: 1.0
ID: 1014396.0, 城市: 2, 区域: 1.0
ID: 1014407.0, 城市: 2, 区域: 1.0
ID: 1014408.0, 城市: 2, 区域: 1.0
ID: 1014409.0, 城市: 2, 区域: 1.0
ID: 1014410.0, 城市: 2, 区域: 1.0
ID: 1014411.0, 城市: 2, 区域: 1.0
ID: 1014412.0, 城市: 2, 区域: 1.0
ID: 1014413.0, 城市: 2, 区域: 1.0
ID: 1014414.0, 城市: 2, 区域: 1.0
ID: 1014415.0, 城市: 2, 区域: 1.0
ID: 1014416.0, 城市: 2, 区域: 1.0
ID: 1014417.0, 城市: 2, 区域: 1.0
ID: 10144

In [47]:
print("="*50)
print("X_test_final 的形状（行数 × 列数）：")
print(f"形状：{X_train.shape}")  # 输出格式：(行数, 列数)，例如 (200, 50) 表示200行样本、50个特征
print(f"- 样本数量（行数）：{X_train.shape[0]}")
print(f"- 特征数量（列数）：{X_train.shape[1]}")

# 2. 打印 X_test_final 的属性（即所有列名，对应自变量特征）
print("\n" + "="*50)
print("X_test_final 的所有属性（列名/特征名）：")
# 方法1：直接打印列名（适合列数少的情况）
print("列名列表：")
print(X_train.columns.tolist())  

X_test_final 的形状（行数 × 列数）：
形状：(103871, 1158)
- 样本数量（行数）：103871
- 特征数量（列数）：1158

X_test_final 的所有属性（列名/特征名）：
列名列表：
['const', '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_西南', '朝向_西北', '朝向_东北', '楼层_低', '楼层_地', '楼层_底', '楼层_顶', '楼层_高', '权属_二类经济适用房', '权属_动迁安置房', '权属_售后公房', '权属_商品房', '权属_回迁房', '权属_央产房', '权属_定向安置房', '权属_已购公房', '权属_房改房', '权属_拆迁还建房', '权属_私产', '权属_经济适用房', '权属_限价商品房', '权属_集资房', '用途_公寓', '用途_公寓/住宅', '用途_公寓/公寓', '用途_公寓（住宅）', '用途_写字楼', '用途_别墅', '用途_商业', '用途_商业办公类', '用途_商住两用', '用途_商务公寓', '用途_商务型公寓', '用途_四合院', '用途_底商', '用途_新式里弄', '用途_普通住宅', '用途_老公寓', '用途_花园洋房', '用途_车库', '用途_酒店式公寓', '别墅_双拼', '别墅_叠拼', '别墅_独栋', '别墅_联排', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '城市_7', '城市_8', '城市_9', '城市_10', '城市_11', '区域_4.0', '区域_5.0', '区域_7.0', '区域_8.0', '区域_9.0', '区域_10.0', '区域_11.0', '区域_12.0', '区域_13.0', '区域_14.0', '区域_15.0', '区域_16.0', '区域_17.0', '区域_18.0', '区域_19.0', '区域_20.0', '区域_21.0', '区域_22.0', '区域_23.0', '区域_24.0', '区域_26.0', '区域_27.0', '区域_28.0', '区域_29.0', '区域_30.0', '区域_31.0', '区域

In [48]:
print("="*50)
print("X_test_final 的形状（行数 × 列数）：")
print(f"形状：{special_rows_df.shape}")  # 输出格式：(行数, 列数)，例如 (200, 50) 表示200行样本、50个特征
print(f"- 样本数量（行数）：{special_rows_df.shape[0]}")
print(f"- 特征数量（列数）：{special_rows_df.shape[1]}")

# 2. 打印 X_test_final 的属性（即所有列名，对应自变量特征）
print("\n" + "="*50)
print("X_test_final 的所有属性（列名/特征名）：")
# 方法1：直接打印列名（适合列数少的情况）
print("列名列表：")
print(special_rows_df.columns.tolist())  

X_test_final 的形状（行数 × 列数）：
形状：(228, 1185)
- 样本数量（行数）：228
- 特征数量（列数）：1185

X_test_final 的所有属性（列名/特征名）：
列名列表：
['const', '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_西南', '朝向_西北', '朝向_东北', '楼层_低', '楼层_地', '楼层_底', '楼层_顶', '楼层_高', '权属_二类经济适用房', '权属_动迁安置房', '权属_售后公房', '权属_商品房', '权属_回迁房', '权属_央产房', '权属_定向安置房', '权属_已购公房', '权属_房改房', '权属_拆迁还建房', '权属_私产', '权属_经济适用房', '权属_限价商品房', '权属_集资房', '用途_公寓', '用途_公寓/住宅', '用途_公寓/公寓', '用途_公寓（住宅）', '用途_写字楼', '用途_别墅', '用途_商业', '用途_商业办公类', '用途_商住两用', '用途_商务公寓', '用途_商务型公寓', '用途_四合院', '用途_底商', '用途_新式里弄', '用途_普通住宅', '用途_老公寓', '用途_花园洋房', '用途_车库', '用途_酒店式公寓', '别墅_双拼', '别墅_叠拼', '别墅_独栋', '别墅_联排', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '城市_7', '城市_8', '城市_9', '城市_10', '城市_11', '区域_4.0', '区域_5.0', '区域_7.0', '区域_8.0', '区域_9.0', '区域_10.0', '区域_11.0', '区域_12.0', '区域_13.0', '区域_14.0', '区域_15.0', '区域_16.0', '区域_17.0', '区域_18.0', '区域_19.0', '区域_20.0', '区域_21.0', '区域_22.0', '区域_23.0', '区域_24.0', '区域_26.0', '区域_27.0', '区域_28.0', '区域_29.0', '区域_30.0', '区域_31.0', '区域_32.0'

In [49]:
# 检测special_rows_df中是否有null值
null_count = special_rows_df.isnull().sum().sum()

if null_count == 0:
    print("special_rows_df没有null值 ✓")
else:
    print(f"special_rows_df有{null_count}个null值")
    # 显示具体哪些列有null值
    null_cols = special_rows_df.isnull().sum()
    null_cols = null_cols[null_cols > 0]
    print("有null值的列:")
    for col, count in null_cols.items():
        print(f"  - {col}: {count}个null")

special_rows_df没有null值 ✓


In [50]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances

def process_special_tuples(
    special_rows_df,        # 所有特殊元组（数据框，包含共同列、测试集特殊列；列名与X_train一致）
    special_cols,           # 特殊列列表（如训练集比测试集多出来的虚拟变量列）
    X_train,                # 训练集（编码后，包含城市/区域/板块虚拟变量）
    df_test=None,           # 可选：原始测试集（未必需要），用于按 ID 查找原始行（如果有）
    id_col='ID',            # ID列名
    city_prefix='城市_',    # 城市虚拟变量前缀
    county_prefix='区域_',  # 区域虚拟变量前缀
    plate_prefix='板块_',   # 板块虚拟变量前缀
    knn_features=None,      # 距离计算特征（默认 ['lon','lat','建筑面积']）
    m=5,                    # 最近邻居数量
    verbose=True            # 是否打印过程
):

    if knn_features is None:
        knn_features = ['lon','lat','建筑面积']

    # 列名集合（训练集中的城市/区域/板块虚拟变量）
    city_cols_train = [col for col in X_train.columns if col.startswith(city_prefix)]
    county_cols_train = [col for col in X_train.columns if col.startswith(county_prefix)]
    plate_cols_train = [col for col in X_train.columns if col.startswith(plate_prefix)]

    # 复制数据以免修改输入
    processed_tuples = special_rows_df.copy()

    # 工具：标准化特征
    def standardize_features(features):
        scaler = StandardScaler()
        return scaler.fit_transform(features), scaler

    # 为后面快速比较，准备训练集中城市/区域的数值矩阵（float）
    X_city_mat = X_train[city_cols_train].fillna(0).astype(float)
    X_county_mat = X_train[county_cols_train].fillna(0).astype(float)

    # 辅助：从 df_test 按 id 查值（兼容若干命名）
    def _get_df_test_value(special_id, candidates):
        if df_test is None or id_col not in df_test.columns:
            return None
        rows = df_test[df_test[id_col] == special_id]
        if rows.shape[0] == 0:
            return None
        valrow = rows.iloc[0]
        for cname in candidates:
            if cname in df_test.columns:
                v = valrow.get(cname, None)
                if pd.isna(v):
                    continue
                return v
        return None

    # ---------- 遍历每个特殊元组 ----------
    for tuple_idx, special_tuple in special_rows_df.iterrows():
        if verbose:
            print("\n" + "="*60)
            print(f"处理特殊元组：索引={tuple_idx}  ID={special_tuple.get(id_col, '无')}")

        # 1) 从 special_cols 中找出本行值为 1 的那些特殊列
        valid_special = []
        for col in special_cols:
            if col not in special_rows_df.columns:
                continue
            val = special_tuple.get(col, 0)
            try:
                val_num = pd.to_numeric(val, errors='coerce')
                if not np.isnan(val_num) and val_num == 1:
                    valid_special.append(col)
            except:
                continue
        if not valid_special:
            if verbose:
                print("  无值为1的特殊列，跳过")
            continue
        if verbose:
            print(f"  有效特殊列（值为1）：{valid_special}")

        # 2) 判断类别：区域/板块/两者都有
        county_special = [c for c in valid_special if c.startswith(county_prefix)]
        plate_special = [c for c in valid_special if c.startswith(plate_prefix)]

        if len(county_special) > 0 and len(plate_special) == 0:
            category = "仅区域特殊列"
        elif len(plate_special) > 0 and len(county_special) == 0:
            category = "仅板块特殊列"
        elif len(county_special) > 0 and len(plate_special) > 0:
            category = "区域和板块均有特殊列"
        else:
            if verbose:
                print("  无有效区域/板块特殊列，跳过")
            continue
        if verbose:
            print(f"  分类：{category}")

        # 3) 计算城市/区域“编码模式” —— 包括全部为0的情况
        tuple_city_vec = special_tuple.reindex(city_cols_train).fillna(0).astype(float)
        tuple_county_vec = special_tuple.reindex(county_cols_train).fillna(0).astype(float)

        city_any_active = (tuple_city_vec > 0).any()
        county_any_active = (tuple_county_vec > 0).any()

        if verbose:
            print(f"  城市编码（前6）：{tuple_city_vec.head(6).to_dict()}  | 任一激活: {city_any_active}")
            print(f"  区域编码（前6）：{tuple_county_vec.head(6).to_dict()}  | 任一激活: {county_any_active}")

        # 4) 构造邻居候选掩码（与原逻辑一致）
        neighbor_mask = pd.Series(True, index=X_train.index)
        city_match_mask = (X_city_mat.eq(tuple_city_vec.values, axis=1)).all(axis=1)

        if category == "仅区域特殊列":
            neighbor_mask &= city_match_mask
            if verbose:
                print("  邻居筛选：城市编码与特殊元组完全相同（包含全部为0的情况）")
        elif category == "仅板块特殊列":
            county_match_mask = (X_county_mat.eq(tuple_county_vec.values, axis=1)).all(axis=1)
            neighbor_mask &= city_match_mask & county_match_mask
            if verbose:
                print("  邻居筛选：城市编码 AND 区域编码 与特殊元组完全相同（包含全部为0的情况）")
        else:  # 区域和板块均有特殊列
            neighbor_mask &= city_match_mask
            if verbose:
                print("  邻居筛选：城市编码相同（区域/板块将从这些邻居中求均值）")

        candidate_neighbors = X_train[neighbor_mask].copy()

        # ---------- 新增逻辑：当候选集为空时，优先根据 df_test 查找并按用户指定规则选择候选 ----------
        if candidate_neighbors.shape[0] == 0:
            if verbose:
                print("  精确匹配无候选邻居，尝试基于 df_test 的策略（若 df_test 有该 ID）")

            special_id = special_tuple.get(id_col, None)

            # 仅区域特殊列：若 df_test 中该行 城市 == 0，则用 X_train 中城市全部为0 的样本
            if category == "仅区域特殊列" and special_id is not None and df_test is not None:
                city_val = _get_df_test_value(special_id, ['城市', 'city'])
                # 将 city_val 转为数值后比较（兼容 int64）
                try:
                    city_val_num = np.nan if city_val is None else float(city_val)
                except:
                    city_val_num = np.nan
                if not np.isnan(city_val_num) and city_val_num == 0:
                    if verbose:
                        print("  df_test 中该 ID 的城市值为 0 → 选择 X_train 中城市全部为0 的样本作为候选")
                    mask = (X_city_mat.sum(axis=1) == 0)
                    candidate_neighbors = X_train[mask].copy()

            # 仅板块特殊列：若 df_test 中该行 区域 == 1，则选择同城市且训练集中区域全为0 的样本
            if candidate_neighbors.shape[0] == 0 and category == "仅板块特殊列" and special_id is not None and df_test is not None:
                area_val = _get_df_test_value(special_id, ['区域', 'area', 'county'])
                try:
                    area_val_num = np.nan if area_val is None else float(area_val)
                except:
                    area_val_num = np.nan
                if not np.isnan(area_val_num) and area_val_num == 1:
                    if verbose:
                        print("  df_test 中该 ID 的区域值为 1 → 选择同城市且训练集中区域全为0 的样本作为候选")
                    # 城市完全相同的掩码（与前面 city_match_mask 一致）
                    same_city_mask = (X_city_mat.eq(tuple_city_vec.values, axis=1)).all(axis=1)
                    county_all_zero_mask = (X_county_mat.sum(axis=1) == 0)
                    combined_mask = same_city_mask & county_all_zero_mask
                    # 若 combined_mask 有样本，则采用
                    if combined_mask.any():
                        candidate_neighbors = X_train[combined_mask].copy()
                    else:
                        if verbose:
                            print("  虽满足 df_test 区域==1，但在训练集中未找到同城市且区域全0 的样本")

        # 若放宽后仍为空，则按原先的宽松策略（保底：city 任一激活或城市全 0）
        if candidate_neighbors.shape[0] == 0:
            if verbose:
                print("  放宽：使用城市任一激活(>0)或区域任一激活匹配（原有回退策略）")
            if city_any_active:
                active_city_cols = [c for c in city_cols_train if tuple_city_vec.get(c, 0) > 0]
                if active_city_cols:
                    mask = (X_train[active_city_cols].sum(axis=1) > 0)
                    candidate_neighbors = X_train[mask].copy()
            else:
                mask = (X_city_mat.sum(axis=1) == 0)
                candidate_neighbors = X_train[mask].copy()

            if candidate_neighbors.shape[0] == 0:
                if verbose:
                    print("  放宽后仍无候选，退化为整个训练集作为候选")
                candidate_neighbors = X_train.copy()

        if verbose:
            print(f"  备选邻居数量：{len(candidate_neighbors)}")

        # 5) 基于 knn_features 做最近 m 个邻居的选择
        missing_feats = [f for f in knn_features if f not in candidate_neighbors.columns or pd.isna(candidate_neighbors[f]).all()]
        if missing_feats:
            if verbose:
                print(f"  距离计算所需特征缺失或全NaN：{missing_feats}，跳过该元组")
            continue

        neighbor_features = candidate_neighbors[knn_features].astype(float).fillna(0.0)
        neighbor_features_scaled, scaler = standardize_features(neighbor_features)
        tuple_features = special_tuple.reindex(knn_features).fillna(0).astype(float).values.reshape(1, -1)
        tuple_features_scaled = scaler.transform(tuple_features)
        distances = pairwise_distances(tuple_features_scaled, neighbor_features_scaled, metric='euclidean')[0]
        candidate_neighbors = candidate_neighbors.copy()
        candidate_neighbors['__distance__'] = distances

        m_actual = min(m, len(candidate_neighbors))
        nearest_neighbors = candidate_neighbors.sort_values('__distance__').head(m_actual)
        if verbose:
            print(f"  选取最近 {m_actual} 个邻居，距离范围：{nearest_neighbors['__distance__'].min():.4f} ~ {nearest_neighbors['__distance__'].max():.4f}")

        # 6) 确定需要计算均值的列（即训练集的县/板块列，排除当前特殊列）
        if category == "仅区域特殊列":
            cols_to_avg = [c for c in county_cols_train if c not in county_special]
            cols_to_drop = county_special
        elif category == "仅板块特殊列":
            cols_to_avg = [c for c in plate_cols_train if c not in plate_special]
            cols_to_drop = plate_special
        else:  # 区域和板块均有特殊列
            cols_to_avg = [c for c in county_cols_train if c not in county_special] + \
                          [c for c in plate_cols_train if c not in plate_special]
            cols_to_drop = county_special + plate_special

        if not cols_to_avg:
            if verbose:
                print("  无需计算均值的列，跳过填充与删除")
            continue

        # 7) 计算邻居均值
        neighbor_means = nearest_neighbors[cols_to_avg].astype(float).mean()
        if verbose:
            preview = neighbor_means.head(6).to_dict()
            print(f"  邻居在待填充列的均值（前6项示例）：{preview}")

        # 8) 将均值写回 processed_tuples（新增列则创建），并删除特殊列
        for col in cols_to_avg:
            if col in processed_tuples.columns:
                processed_tuples.at[tuple_idx, col] = neighbor_means.get(col, 0.0)
            else:
                processed_tuples[col] = 0.0
                processed_tuples.at[tuple_idx, col] = neighbor_means.get(col, 0.0)

        if verbose:
            print(f"  填充了 {len(cols_to_avg)} 列；删除了 {len(cols_to_drop)} 个特殊列（若存在）")

    # ---------- 结束：打印与返回 ----------
    if verbose:
        print("\n" + "="*70)
        print("所有特殊元组处理完成")
        print(f"处理后特殊元组数量：{len(processed_tuples)}")
        preview_cols = [id_col] if id_col in processed_tuples.columns else []
        preview_cols += [c for c in processed_tuples.columns if c.startswith((county_prefix, plate_prefix))][:6]
        if preview_cols:
            print("处理后前3行关键列预览：")
            print(processed_tuples[preview_cols].head(3).to_string(index=True))

    return processed_tuples


In [51]:
import pandas as pd

def process_test(X_train, X_test, id_col='ID'):
    # 原有处理逻辑（省略，同上）
    # ...

    # 检查X_test_cleaned空值
    has_null = X_test_cleaned.isnull().any().any()
    null_stats = X_test_cleaned.isnull().sum()[X_test_cleaned.isnull().sum() > 0].to_dict()

    return X_test_cleaned, special_rows_df, special_cols, has_null, null_stats
X_test_cleaned, special_rows_df, special_cols, has_null, null_stats = process_test(X_train, X_test)
print("是否有缺失值：", has_null)
print("缺失值统计：", null_stats)  # 键为列名，值为缺失数量


是否有缺失值： False
缺失值统计： {}


### 处理228个特殊元组

In [52]:
print(special_rows_df.shape)

(228, 1185)


In [53]:
processed_result = process_special_tuples(special_rows_df=special_rows_df,
special_cols=special_cols,
X_train=X_train,
id_col='ID',m=5,verbose=True)


处理特殊元组：索引=7047  ID=1007047.0
  有效特殊列（值为1）：['板块_144.0']
  分类：仅板块特殊列
  城市编码（前6）：{'城市_1': 0.0, '城市_2': 0.0, '城市_3': 0.0, '城市_4': 0.0, '城市_5': 0.0, '城市_6': 0.0}  | 任一激活: False
  区域编码（前6）：{'区域_4.0': 0.0, '区域_5.0': 0.0, '区域_7.0': 0.0, '区域_8.0': 0.0, '区域_9.0': 0.0, '区域_10.0': 0.0}  | 任一激活: True
  邻居筛选：城市编码 AND 区域编码 与特殊元组完全相同（包含全部为0的情况）
  备选邻居数量：434
  选取最近 5 个邻居，距离范围：0.5130 ~ 0.5538
  邻居在待填充列的均值（前6项示例）：{'板块_2.0': 0.0, '板块_3.0': 0.0, '板块_4.0': 0.0, '板块_5.0': 0.0, '板块_6.0': 0.0, '板块_8.0': 0.0}
  填充了 976 列；删除了 1 个特殊列（若存在）

处理特殊元组：索引=9491  ID=1009491.0
  有效特殊列（值为1）：['板块_77.0']
  分类：仅板块特殊列
  城市编码（前6）：{'城市_1': 0.0, '城市_2': 1.0, '城市_3': 0.0, '城市_4': 0.0, '城市_5': 0.0, '城市_6': 0.0}  | 任一激活: True
  区域编码（前6）：{'区域_4.0': 0.0, '区域_5.0': 0.0, '区域_7.0': 0.0, '区域_8.0': 0.0, '区域_9.0': 0.0, '区域_10.0': 0.0}  | 任一激活: True
  邻居筛选：城市编码 AND 区域编码 与特殊元组完全相同（包含全部为0的情况）
  备选邻居数量：6598


c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


  选取最近 5 个邻居，距离范围：0.0023 ~ 0.0167
  邻居在待填充列的均值（前6项示例）：{'板块_2.0': 0.0, '板块_3.0': 0.0, '板块_4.0': 0.0, '板块_5.0': 0.0, '板块_6.0': 0.0, '板块_8.0': 0.0}
  填充了 976 列；删除了 1 个特殊列（若存在）

处理特殊元组：索引=10803  ID=1010803.0
  有效特殊列（值为1）：['板块_848.0']
  分类：仅板块特殊列
  城市编码（前6）：{'城市_1': 0.0, '城市_2': 1.0, '城市_3': 0.0, '城市_4': 0.0, '城市_5': 0.0, '城市_6': 0.0}  | 任一激活: True
  区域编码（前6）：{'区域_4.0': 0.0, '区域_5.0': 0.0, '区域_7.0': 0.0, '区域_8.0': 0.0, '区域_9.0': 0.0, '区域_10.0': 0.0}  | 任一激活: True
  邻居筛选：城市编码 AND 区域编码 与特殊元组完全相同（包含全部为0的情况）
  备选邻居数量：4990
  选取最近 5 个邻居，距离范围：0.1976 ~ 0.3340
  邻居在待填充列的均值（前6项示例）：{'板块_2.0': 0.0, '板块_3.0': 0.0, '板块_4.0': 0.0, '板块_5.0': 0.0, '板块_6.0': 0.0, '板块_8.0': 0.0}


c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\heart\AppData\Local\Temp\ipykernel_15560\1997106155.py:238: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  processed_tuples.at[tuple_idx, col] = neighbor_means.get(col, 0.0)
C:\Users\heart\AppData\Local\Temp\ipykernel_15560\1997106155.py:238: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  processed_tuples.at[tuple_idx, col] = neighbor_means.get(col, 0.0)
c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid fea

  填充了 976 列；删除了 1 个特殊列（若存在）

处理特殊元组：索引=11661  ID=1011661.0
  有效特殊列（值为1）：['板块_358.0']
  分类：仅板块特殊列
  城市编码（前6）：{'城市_1': 0.0, '城市_2': 1.0, '城市_3': 0.0, '城市_4': 0.0, '城市_5': 0.0, '城市_6': 0.0}  | 任一激活: True
  区域编码（前6）：{'区域_4.0': 0.0, '区域_5.0': 0.0, '区域_7.0': 0.0, '区域_8.0': 0.0, '区域_9.0': 0.0, '区域_10.0': 0.0}  | 任一激活: True
  邻居筛选：城市编码 AND 区域编码 与特殊元组完全相同（包含全部为0的情况）
  备选邻居数量：747
  选取最近 5 个邻居，距离范围：0.0026 ~ 0.0077
  邻居在待填充列的均值（前6项示例）：{'板块_2.0': 0.0, '板块_3.0': 0.0, '板块_4.0': 0.0, '板块_5.0': 0.0, '板块_6.0': 0.0, '板块_8.0': 0.0}
  填充了 976 列；删除了 1 个特殊列（若存在）

处理特殊元组：索引=11681  ID=1011681.0
  有效特殊列（值为1）：['板块_358.0']
  分类：仅板块特殊列
  城市编码（前6）：{'城市_1': 0.0, '城市_2': 1.0, '城市_3': 0.0, '城市_4': 0.0, '城市_5': 0.0, '城市_6': 0.0}  | 任一激活: True
  区域编码（前6）：{'区域_4.0': 0.0, '区域_5.0': 0.0, '区域_7.0': 0.0, '区域_8.0': 0.0, '区域_9.0': 0.0, '区域_10.0': 0.0}  | 任一激活: True
  邻居筛选：城市编码 AND 区域编码 与特殊元组完全相同（包含全部为0的情况）
  备选邻居数量：747
  选取最近 5 个邻居，距离范围：0.0021 ~ 0.0063
  邻居在待填充列的均值（前6项示例）：{'板块_2.0': 0.0, '板块_3.0': 0.0, '板块_4.0': 0.0, '板块_5.0':

...

c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


  备选邻居数量：1250
  选取最近 5 个邻居，距离范围：0.1533 ~ 0.2447
  邻居在待填充列的均值（前6项示例）：{'板块_2.0': 0.0, '板块_3.0': 0.0, '板块_4.0': 0.0, '板块_5.0': 0.0, '板块_6.0': 0.0, '板块_8.0': 0.0}
  填充了 976 列；删除了 1 个特殊列（若存在）

处理特殊元组：索引=33457  ID=1033457.0
  有效特殊列（值为1）：['板块_699.0']
  分类：仅板块特殊列
  城市编码（前6）：{'城市_1': 0.0, '城市_2': 0.0, '城市_3': 0.0, '城市_4': 0.0, '城市_5': 0.0, '城市_6': 0.0}  | 任一激活: True
  区域编码（前6）：{'区域_4.0': 0.0, '区域_5.0': 0.0, '区域_7.0': 0.0, '区域_8.0': 0.0, '区域_9.0': 0.0, '区域_10.0': 0.0}  | 任一激活: True
  邻居筛选：城市编码 AND 区域编码 与特殊元组完全相同（包含全部为0的情况）
  备选邻居数量：1250
  选取最近 5 个邻居，距离范围：0.1539 ~ 0.2059
  邻居在待填充列的均值（前6项示例）：{'板块_2.0': 0.0, '板块_3.0': 0.0, '板块_4.0': 0.0, '板块_5.0': 0.0, '板块_6.0': 0.0, '板块_8.0': 0.0}
  填充了 976 列；删除了 1 个特殊列（若存在）

处理特殊元组：索引=33566  ID=1033566.0
  有效特殊列（值为1）：['板块_1056.0']
  分类：仅板块特殊列
  城市编码（前6）：{'城市_1': 0.0, '城市_2': 0.0, '城市_3': 0.0, '城市_4': 0.0, '城市_5': 0.0, '城市_6': 0.0}  | 任一激活: True
  区域编码（前6）：{'区域_4.0': 0.0, '区域_5.0': 0.0, '区域_7.0': 0.0, '区域_8.0': 0.0, '区域_9.0': 0.0, '区域_10.0': 0.0}  | 任一激活: True
  邻

c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


  选取最近 5 个邻居，距离范围：0.4528 ~ 0.4597
  邻居在待填充列的均值（前6项示例）：{'板块_2.0': 0.0, '板块_3.0': 0.0, '板块_4.0': 0.0, '板块_5.0': 0.0, '板块_6.0': 0.0, '板块_8.0': 0.0}
  填充了 976 列；删除了 1 个特殊列（若存在）

处理特殊元组：索引=33620  ID=1033620.0
  有效特殊列（值为1）：['板块_1056.0']
  分类：仅板块特殊列
  城市编码（前6）：{'城市_1': 0.0, '城市_2': 0.0, '城市_3': 0.0, '城市_4': 0.0, '城市_5': 0.0, '城市_6': 0.0}  | 任一激活: True
  区域编码（前6）：{'区域_4.0': 0.0, '区域_5.0': 0.0, '区域_7.0': 0.0, '区域_8.0': 0.0, '区域_9.0': 0.0, '区域_10.0': 0.0}  | 任一激活: True
  邻居筛选：城市编码 AND 区域编码 与特殊元组完全相同（包含全部为0的情况）
  备选邻居数量：3392
  选取最近 5 个邻居，距离范围：0.4532 ~ 0.4598
  邻居在待填充列的均值（前6项示例）：{'板块_2.0': 0.0, '板块_3.0': 0.0, '板块_4.0': 0.0, '板块_5.0': 0.0, '板块_6.0': 0.0, '板块_8.0': 0.0}
  填充了 976 列；删除了 1 个特殊列（若存在）

处理特殊元组：索引=33707  ID=1033707.0
  有效特殊列（值为1）：['板块_699.0']
  分类：仅板块特殊列
  城市编码（前6）：{'城市_1': 0.0, '城市_2': 0.0, '城市_3': 0.0, '城市_4': 0.0, '城市_5': 0.0, '城市_6': 0.0}  | 任一激活: True
  区域编码（前6）：{'区域_4.0': 0.0, '区域_5.0': 0.0, '区域_7.0': 0.0, '区域_8.0': 0.0, '区域_9.0': 0.0, '区域_10.0': 0.0}  | 任一激活: True
  邻居筛选：城市编码 AND 区

c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\heart\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [54]:
processed_result
print("="*50)
print("X_test_final 的形状（行数 × 列数）：")
print(f"形状：{processed_result.shape}")  # 输出格式：(行数, 列数)，例如 (200, 50) 表示200行样本、50个特征
print(f"- 样本数量（行数）：{processed_result.shape[0]}")
print(f"- 特征数量（列数）：{processed_result.shape[1]}")

# 2. 打印 X_test_final 的属性（即所有列名，对应自变量特征）
print("\n" + "="*50)
print("X_test_final 的所有属性（列名/特征名）：")
# 方法1：直接打印列名（适合列数少的情况）
print("列名列表：")
print(processed_result.columns.tolist())  


X_test_final 的形状（行数 × 列数）：
形状：(228, 1185)
- 样本数量（行数）：228
- 特征数量（列数）：1185

X_test_final 的所有属性（列名/特征名）：
列名列表：
['const', '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_西南', '朝向_西北', '朝向_东北', '楼层_低', '楼层_地', '楼层_底', '楼层_顶', '楼层_高', '权属_二类经济适用房', '权属_动迁安置房', '权属_售后公房', '权属_商品房', '权属_回迁房', '权属_央产房', '权属_定向安置房', '权属_已购公房', '权属_房改房', '权属_拆迁还建房', '权属_私产', '权属_经济适用房', '权属_限价商品房', '权属_集资房', '用途_公寓', '用途_公寓/住宅', '用途_公寓/公寓', '用途_公寓（住宅）', '用途_写字楼', '用途_别墅', '用途_商业', '用途_商业办公类', '用途_商住两用', '用途_商务公寓', '用途_商务型公寓', '用途_四合院', '用途_底商', '用途_新式里弄', '用途_普通住宅', '用途_老公寓', '用途_花园洋房', '用途_车库', '用途_酒店式公寓', '别墅_双拼', '别墅_叠拼', '别墅_独栋', '别墅_联排', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '城市_7', '城市_8', '城市_9', '城市_10', '城市_11', '区域_4.0', '区域_5.0', '区域_7.0', '区域_8.0', '区域_9.0', '区域_10.0', '区域_11.0', '区域_12.0', '区域_13.0', '区域_14.0', '区域_15.0', '区域_16.0', '区域_17.0', '区域_18.0', '区域_19.0', '区域_20.0', '区域_21.0', '区域_22.0', '区域_23.0', '区域_24.0', '区域_26.0', '区域_27.0', '区域_28.0', '区域_29.0', '区域_30.0', '区域_31.0', '区域_32.0'

In [55]:
print("="*50)
print("X_test_final 的形状（行数 × 列数）：")
print(f"形状：{X_train.shape}")  # 输出格式：(行数, 列数)，例如 (200, 50) 表示200行样本、50个特征
print(f"- 样本数量（行数）：{X_train.shape[0]}")
print(f"- 特征数量（列数）：{X_train.shape[1]}")

# 2. 打印 X_test_final 的属性（即所有列名，对应自变量特征）
print("\n" + "="*50)
print("X_test_final 的所有属性（列名/特征名）：")
# 方法1：直接打印列名（适合列数少的情况）
print("列名列表：")
print(X_train.columns.tolist())  


X_test_final 的形状（行数 × 列数）：
形状：(103871, 1158)
- 样本数量（行数）：103871
- 特征数量（列数）：1158

X_test_final 的所有属性（列名/特征名）：
列名列表：
['const', '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_西南', '朝向_西北', '朝向_东北', '楼层_低', '楼层_地', '楼层_底', '楼层_顶', '楼层_高', '权属_二类经济适用房', '权属_动迁安置房', '权属_售后公房', '权属_商品房', '权属_回迁房', '权属_央产房', '权属_定向安置房', '权属_已购公房', '权属_房改房', '权属_拆迁还建房', '权属_私产', '权属_经济适用房', '权属_限价商品房', '权属_集资房', '用途_公寓', '用途_公寓/住宅', '用途_公寓/公寓', '用途_公寓（住宅）', '用途_写字楼', '用途_别墅', '用途_商业', '用途_商业办公类', '用途_商住两用', '用途_商务公寓', '用途_商务型公寓', '用途_四合院', '用途_底商', '用途_新式里弄', '用途_普通住宅', '用途_老公寓', '用途_花园洋房', '用途_车库', '用途_酒店式公寓', '别墅_双拼', '别墅_叠拼', '别墅_独栋', '别墅_联排', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '城市_7', '城市_8', '城市_9', '城市_10', '城市_11', '区域_4.0', '区域_5.0', '区域_7.0', '区域_8.0', '区域_9.0', '区域_10.0', '区域_11.0', '区域_12.0', '区域_13.0', '区域_14.0', '区域_15.0', '区域_16.0', '区域_17.0', '区域_18.0', '区域_19.0', '区域_20.0', '区域_21.0', '区域_22.0', '区域_23.0', '区域_24.0', '区域_26.0', '区域_27.0', '区域_28.0', '区域_29.0', '区域_30.0', '区域_31.0', '区域

### 测试集有228行的区域/板块是训练集没有的

# 注意按地理层级寻找邻居时，有可能特殊元组城市取值为1/区域取值为1，都会由于虚拟变量为避免陷阱找不到邻居，要特殊处理

#### 将修改好的特殊元组放回测试集矩阵

processed_result无空值，包含ID,const，但是多26个特殊列

In [56]:
def update_X_test_with_processed_result(X_test_cleaned, processed_result, id_col='ID'):

    if id_col not in X_test_cleaned.columns:
        raise ValueError(f"X_test_cleaned中不存在ID列 '{id_col}'")
    if id_col not in processed_result.columns:
        raise ValueError(f"processed_result中不存在ID列 '{id_col}'")
    
    # 创建副本以避免修改原始数据
    X_test_updated = X_test_cleaned.copy()
    
    # 遍历processed_result的每一行
    for idx, processed_row in processed_result.iterrows():
        processed_id = processed_row[id_col]
        
        # 在X_test_cleaned中查找相同ID的行
        matching_indices = X_test_updated[X_test_updated[id_col] == processed_id].index
        
        if len(matching_indices) == 0:
            print(f"警告: 在X_test_cleaned中未找到ID为 {processed_id} 的行")
            continue
        
        if len(matching_indices) > 1:
            print(f"警告: 在X_test_cleaned中找到多个ID为 {processed_id} 的行，将更新第一个")
        
        # 获取X_test_cleaned中匹配行的索引
        target_idx = matching_indices[0]
        
        # 遍历X_test_cleaned的每一列（除了ID列）
        for col in X_test_updated.columns:
            if col != id_col and col in processed_result.columns:
                # 用processed_result的值更新X_test_cleaned的对应列
                X_test_updated.loc[target_idx, col] = processed_row[col]
    
    print(f"更新完成: 处理了 {len(processed_result)} 行特殊元组")
    return X_test_updated

def convert_virtual_to_float(test_df):
    # 筛选区域/板块开头的列，转为float
    target_cols = [col for col in test_df.columns if col.startswith(("区域_", "板块_"))]
    test_df[target_cols] = test_df[target_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)
    return test_df
X_test_cleaned = convert_virtual_to_float(X_test)

### 原始测试数据的虚拟变量变为float类型

In [57]:
print("="*50)
print("X_test_final 的形状（行数 × 列数）：")
print(f"形状：{X_test.shape}")  # 输出格式：(行数, 列数)，例如 (200, 50) 表示200行样本、50个特征
print(f"- 样本数量（行数）：{X_test.shape[0]}")
print(f"- 特征数量（列数）：{X_test.shape[1]}")

# 2. 打印 X_test_final 的属性（即所有列名，对应自变量特征）
print("\n" + "="*50)
print("X_test_final 的所有属性（列名/特征名）：")
# 方法1：直接打印列名（适合列数少的情况）
print("列名列表：")
print(X_test.columns.tolist())  


X_test_final 的形状（行数 × 列数）：
形状：(34017, 1185)
- 样本数量（行数）：34017
- 特征数量（列数）：1185

X_test_final 的所有属性（列名/特征名）：
列名列表：
['ID', '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_西南', '朝向_西北', '朝向_东北', '楼层_低', '楼层_地', '楼层_底', '楼层_顶', '楼层_高', '用途_公寓/住宅', '用途_公寓/公寓', '用途_公寓（住宅）', '用途_别墅', '用途_商业', '用途_商业办公类', '用途_商住两用', '用途_商务公寓', '用途_商务型公寓', '用途_新式里弄', '用途_普通住宅', '用途_老公寓', '用途_花园洋房', '用途_车库', '用途_酒店式公寓', '权属_二类经济适用房', '权属_动迁安置房', '权属_售后公房', '权属_商品房', '权属_央产房', '权属_定向安置房', '权属_已购公房', '权属_房改房', '权属_拆迁还建房', '权属_私产', '权属_经济适用房', '权属_限价商品房', '权属_集资房', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '城市_7', '城市_8', '城市_9', '城市_10', '城市_11', '区域_3.0', '区域_4.0', '区域_5.0', '区域_7.0', '区域_8.0', '区域_9.0', '区域_10.0', '区域_11.0', '区域_12.0', '区域_13.0', '区域_14.0', '区域_15.0', '区域_16.0', '区域_17.0', '区域_18.0', '区域_19.0', '区域_20.0', '区域_21.0', '区域_22.0', '区域_23.0', '区域_24.0', '区域_26.0', '区域_27.0', '区域_28.0', '区域_29.0', '区域_30.0', '区域_32.0', '区域_33.0', '区域_34.0', '区域_35.0', '区域_36.0', '区域_37.0', '区域_38.0', '区域_39.0', '区域_4

print("cleaned shape:", X_test_cleaned.shape)

In [58]:
print("cleaned shape:", X_test_cleaned.shape)

cleaned shape: (34017, 1159)


In [59]:
# 调用函数按ID更新测试集
X_test_final = update_X_test_with_processed_result(
    X_test_cleaned=X_test_cleaned,
    processed_result=processed_result,
    id_col='ID',  # 确保与你的ID列名一致
)

C:\Users\heart\AppData\Local\Temp\ipykernel_15560\3989888693.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  X_test_updated.loc[target_idx, col] = processed_row[col]
C:\Users\heart\AppData\Local\Temp\ipykernel_15560\3989888693.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  X_test_updated.loc[target_idx, col] = processed_row[col]
C:\Users\heart\AppData\Local\Temp\ipykernel_15560\3989888693.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  X_test_update

更新完成: 处理了 228 行特殊元组


### 检查测试集

In [60]:
# 假设ID列名为'ID'，直接删除
X_test_final.drop(columns='ID', inplace=True)

In [61]:
if 'const' in X_test_final.columns:
    X_test_final['const'] = 1  # 整列赋值为1

### 测试集使用OLS

In [62]:
# 获取训练集和测试集的列名
train_cols = X_train.columns.tolist()
test_cols = X_test_final.columns.tolist()

print(f"训练集列数: {len(train_cols)}")
print(f"测试集列数: {len(test_cols)}")


训练集列数: 1158
测试集列数: 1158


In [63]:

# 找出测试集中比训练集多的列
extra_cols = list(set(test_cols) - set(train_cols))
print(f"\n测试集比训练集多的列数: {len(extra_cols)}")


测试集比训练集多的列数: 0


In [64]:
def check_X_test_final(X_test_final, key_cols=['ID', 'lon', 'lat', '建筑面积']):
    # 1. 数据形状
    print(f"1. 数据形状（行×列）：{X_test_final.shape}")
    
    # 2. 空值统计
    null_total = X_test_final.isnull().sum().sum()
    null_cols = X_test_final.isnull().sum()[X_test_final.isnull().sum() > 0]
    print(f"2. 空值总数：{null_total}")
    if not null_cols.empty:
        print(f"   含空值的列：{dict(null_cols)}")
    
    # 3. 区县/板块列类型（确认是否为float）
    virtual_cols = [col for col in X_test_final.columns if col.startswith(("区县_", "板块_"))]
    if virtual_cols:
        col_types = X_test_final[virtual_cols].dtypes.unique()
        print(f"3. 区县/板块列类型：{col_types}（需为float64）")
    
    # 4. 关键列存在性（如ID、经纬度、面积）
    missing_key = [col for col in key_cols if col not in X_test_final.columns]
    print(f"4. 关键列存在性：{'全部存在' if not missing_key else f'缺失：{missing_key}'}")

# 调用检查
check_X_test_final(X_test_final)

1. 数据形状（行×列）：(34017, 1158)
2. 空值总数：0
3. 区县/板块列类型：[dtype('int32') dtype('float64') dtype('int64')]（需为float64）
4. 关键列存在性：缺失：['ID']


In [65]:
y_pred_log = model.predict(X_test_final)

y_pred = np.exp(y_pred_log)

In [66]:
print(y_pred.head(10))

0    3.367157e+07
1    2.694527e+06
2    4.004913e+06
3    3.036648e+06
4    9.370408e+06
5    2.636926e+06
6    1.804389e+07
7    3.955118e+06
8    5.252168e+06
9    9.682241e+06
dtype: float64


In [67]:
# 1. 先将预测价格添加到测试集df_test中（确保y_pred是你的预测结果）
df_test['Price'] = y_pred  # y_pred是模型输出的预测值，比如之前的y_pred变量

# 2. 同时选择ID和预测价格列，保存为CSV
df_test[['ID', 'Price']].to_csv("price_predicted.csv", index=False)

print("\n✅ 已完成预测，结果保存至 price_predicted.csv")


✅ 已完成预测，结果保存至 price_predicted.csv


In [ ]:
import pandas as pd
import os

base_file = 'ruc_Class25Q2_test_price_predicted.csv'
src_file = 'rent_predicted_updated.csv'

# 1. 读取基底数据（优先用 submit1027.csv，没有就再看目标文件是否存在）
dst_file = 'submit1027.csv'
if os.path.exists(base_file):
    base = pd.read_csv(base_file)
else:
    base = pd.DataFrame(columns=['ID', '预测价格'])   # 空表头，确保后面 concat 不出错

# 2. 读取待追加的 ID/Price
new = pd.read_csv(src_file, usecols=['ID', 'Price'])
new.columns = ['ID', '预测价格']          # 统一列名

# 3. 拼接
out = pd.concat([base, new], axis=0, ignore_index=True)

# 4. 写出
out.to_csv(dst_file, index=False)
print(f'已完成：共 {len(out)} 条记录 -> {dst_file}')

已完成：共 43790 条记录 -> submit1027.csv


In [69]:
import pandas as pd
import os

def merge_price_rent_csv(
    price_file_path,   # price_predicted.csv文件路径
    rent_file_path,    # rent_predicted_updated.csv文件路径
    output_file_path   # 输出文件1028submi.csv路径
):
    """
    合并price_predicted.csv和rent_predicted_updated.csv，生成仅含ID、Price两列的1028submi.csv
    """
    # ---------------- 1. 校验文件是否存在 ----------------
    if not os.path.exists(price_file_path):
        raise FileNotFoundError(f"price_predicted.csv文件未找到，请检查路径：{price_file_path}")
    if not os.path.exists(rent_file_path):
        raise FileNotFoundError(f"rent_predicted.csv文件未找到，请检查路径：{rent_file_path}")

    # ---------------- 2. 读取两个CSV文件（仅保留ID和Price列） ----------------
    try:
        # 读取房价预测文件，强制保留ID和Price列（避免多余列干扰）
        df_price = pd.read_csv(
            price_file_path,
            usecols=['ID', 'Price'],  # 只读取需要的两列
            dtype={'ID': str}         # ID设为字符串，避免数字类型不一致（如123 vs "123"）
        )
        # 读取租金预测文件，同样强制保留目标列
        df_rent = pd.read_csv(
            rent_file_path,
            usecols=['ID', 'Price'],
            dtype={'ID': str}
        )
    except KeyError as e:
        # 捕获列名缺失异常（如文件中没有ID或Price列）
        raise ValueError(f"文件缺少必要列{str(e)}，请确保两个CSV都包含'ID'和'Price'列")
    except Exception as e:
        raise RuntimeError(f"读取文件时出错：{str(e)}")

    # ---------------- 3. 纵向合并数据（整合所有ID和Price） ----------------
    # axis=0表示纵向拼接（上下叠加），ignore_index=True重置行索引（避免重复索引）
    df_merged = pd.concat([df_price, df_rent], axis=0, ignore_index=True)

    # ---------------- 4. 可选：检查数据有效性（如重复ID提示） ----------------
    duplicate_ids = df_merged[df_merged.duplicated(subset=['ID'], keep=False)]
    if not duplicate_ids.empty:
        print(f"⚠️  提示：合并后存在{len(duplicate_ids['ID'].unique())}个重复ID（如{duplicate_ids['ID'].iloc[0]}），请确认是否需要去重")

    # ---------------- 5. 保存为目标文件1028submi.csv ----------------
    try:
        df_merged.to_csv(
            output_file_path,
            index=False,          # 不保留行索引（避免多余列）
            encoding='utf-8-sig'  # 兼容中文编码，避免乱码
        )
        print(f"✅ 文件合并成功！目标文件已保存至：{output_file_path}")
        print(f"📊 合并后数据概况：共{len(df_merged)}行数据，包含{len(df_merged['ID'].unique())}个唯一ID")
    except Exception as e:
        raise RuntimeError(f"保存文件时出错：{str(e)}")

    return df_merged  # 可选返回合并后的DataFrame，方便后续处理


# ---------------- 使用示例 ----------------
if __name__ == "__main__":
    # 请根据你的文件实际路径修改以下参数（相对路径或绝对路径均可）
    PRICE_FILE = "price_predicted.csv"       # price_predicted.csv的路径
    RENT_FILE = "rent_predicted_updated.csv" # rent_predicted_updated.csv的路径
    OUTPUT_FILE = "1029submi.csv"            # 输出文件1028submi.csv的路径

    try:
        # 调用函数执行合并
        merged_df = merge_price_rent_csv(
            price_file_path=PRICE_FILE,
            rent_file_path=RENT_FILE,
            output_file_path=OUTPUT_FILE
        )
    except Exception as e:
        print(f"❌ 执行失败：{str(e)}")

✅ 文件合并成功！目标文件已保存至：1029submi.csv
📊 合并后数据概况：共43790行数据，包含43790个唯一ID


In [ ]:
import pandas as pd

# 读取文件
df = pd.read_csv("1029submi.csv")

# 筛选预测价格列有空值的行，提取对应的ID
null_rows = df[df['Price'].isna()]
null_ids = null_rows['ID'].tolist()  # 空值行的ID列表
null_count = len(null_ids)  # 空值数量

# 填充空值为1000000
df['Price'] = df['Price'].fillna(2000000)

# 保存修改后的文件（修正拼写错误：index=False）
df.to_csv("1029submi.csv", index=False)

# 输出结果
print(f"处理完成！预测价格列共填充了 {null_count} 个空值（填充值：1000000）")
if null_count > 0:
    print(f"空值对应的ID为：{null_ids}")
else:
    print("无空值行")

处理完成！预测价格列共填充了 0 个空值（填充值：1000000）
无空值行


# Lasso

### 划分数据集

In [70]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

# ---------------- 0. 数据命名修正（关键：区分全量数据和划分后的子集） ----------------
# 你的原始数据：X_train是全量自变量，y是全量因变量（log(Price)）
# 为避免混淆，先重命名为“全量数据”
X_all = X_train  # 全量自变量（含常数项）
y_all = y        # 全量因变量（log(Price)）

# ---------------- 1. 划分训练集和测试集（80%训练，20%测试） ----------------
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(
    X_all,               # 全量自变量
    y_all,               # 全量因变量
    test_size=0.2,       # 测试集占比20%
    random_state=111     # 固定随机种子，确保划分可复现
)


In [71]:
# ---------------- 2. 训练Lasso模型（带L1正则化） ----------------
# 初始化模型：alpha为正则化强度（>0启用正则化）
# 因X_all含常数项，设置fit_intercept=False避免重复拟合截距
lasso_model = Lasso(
    alpha=0.1,          # 正则化强度（可调整，如0.1、1等）
    fit_intercept=False, # 不额外拟合截距（X已含常数项）
    random_state=111     # 固定随机种子
)

# 用划分后的训练集拟合模型
lasso_model.fit(X_train_split, y_train_split)


Lasso(alpha=0.1, fit_intercept=False, random_state=111)

In [72]:
# ---------------- 3. 预测与性能评估（转换为原始房价水平） ----------------
# 核心：因变量是log(Price)，需用np.exp()转换为原始房价后计算误差

# 3.1 样本内性能（用划分后的训练集评估）
# 预测log值并转换为原始房价
y_train_pred_log = lasso_model.predict(X_train_split)
y_train_pred = np.exp(y_train_pred_log)  # 预测原始房价
y_train_actual = np.exp(y_train_split)   # 实际原始房价

# 计算样本内MAE和RMAE
mae_train = mean_absolute_error(y_train_actual, y_train_pred)
y_train_mean = y_train_actual.mean()     # 训练集实际房价的平均值
rmae_train = mae_train / y_train_mean    # RMAE = MAE / 实际均值（相对误差）

# 3.2 样本外性能（用划分后的测试集评估）
y_test_pred_log = lasso_model.predict(X_test_split)
y_test_pred = np.exp(y_test_pred_log)    # 预测原始房价
y_test_actual = np.exp(y_test_split)     # 实际原始房价

# 计算样本外MAE和RMAE
mae_test = mean_absolute_error(y_test_actual, y_test_pred)
y_test_mean = y_test_actual.mean()       # 测试集实际房价的平均值
rmae_test = mae_test / y_test_mean       # 相对误差

In [73]:
# ---------------- 6折交叉验证性能（用全量数据评估稳定性） ----------------
# 自定义评分函数：基于原始房价计算MAE和RMAE
def score_mae_raw(model, X, y):
    """自定义MAE评分（原始房价）"""
    y_pred_log = model.predict(X)
    y_pred_raw = np.exp(y_pred_log)
    y_actual_raw = np.exp(y)
    return mean_absolute_error(y_actual_raw, y_pred_raw)

def score_rmae_raw(model, X, y):
    """自定义RMAE评分（原始房价：MAE / 实际均值）"""
    y_pred_log = model.predict(X)
    y_pred_raw = np.exp(y_pred_log)
    y_actual_raw = np.exp(y)
    mae = mean_absolute_error(y_actual_raw, y_pred_raw)
    y_mean = y_actual_raw.mean()  # 该折数据的实际均值
    return mae / y_mean

# 6折交叉验证（全量数据上的平均性能）
cv_mae = cross_val_score(
    lasso_model, X_all, y_all,
    cv=6,
    scoring=score_mae_raw
).mean()

cv_rmae = cross_val_score(
    lasso_model, X_all, y_all,
    cv=6,
    scoring=score_rmae_raw
).mean()

# ---------------- 5. 打印性能报告 ----------------
print("="*70)
print("Lasso回归模型性能报告（原始住房价格水平）")
print("="*70)
print(f"数据划分：全量样本数={len(X_all)} | 训练集={len(X_train_split)}（80%） | 测试集={len(X_test_split)}（20%）")
print(f"正则化强度（alpha）：{lasso_model.alpha}")
print("-"*70)

print("\n【1. 样本内性能（划分后的训练集）】")
print(f"MAE（平均绝对误差）：{mae_train:.4f}")  # 单位：原始房价单位（如元）
print(f"RMAE（相对平均绝对误差）：{rmae_train:.4f}（误差占训练集实际均值的比例）")

print("\n【2. 样本外性能（划分后的测试集）】")
print(f"MAE（平均绝对误差）：{mae_test:.4f}")
print(f"RMAE（相对平均绝对误差）：{rmae_test:.4f}（误差占测试集实际均值的比例）")

print("\n【3. 6折交叉验证性能（全量数据）】")
print(f"平均MAE：{cv_mae:.4f}")
print(f"平均RMAE：{cv_rmae:.4f}（误差占全量数据实际均值的平均比例）")
print("\n" + "="*70)

Lasso回归模型性能报告（原始住房价格水平）
数据划分：全量样本数=103871 | 训练集=83096（80%） | 测试集=20775（20%）
正则化强度（alpha）：0.1
----------------------------------------------------------------------

【1. 样本内性能（划分后的训练集）】
MAE（平均绝对误差）：1306618.3762
RMAE（相对平均绝对误差）：0.5757（误差占训练集实际均值的比例）

【2. 样本外性能（划分后的测试集）】
MAE（平均绝对误差）：1265975.1845
RMAE（相对平均绝对误差）：0.5667（误差占测试集实际均值的比例）

【3. 6折交叉验证性能（全量数据）】
平均MAE：1731136.2063
平均RMAE：0.7452（误差占全量数据实际均值的平均比例）



## OLS性能

In [78]:
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# ---------------- 1. 拟合OLS模型（添加常数项） ----------------
X_train_const = sm.add_constant(X_train_split)
model = sm.OLS(y_train_split, X_train_const).fit()

# ---------------- 2. 样本内性能（训练集） ----------------
y_pred_train = model.predict(X_train_const)
mae_in = mean_absolute_error(y_train_split, y_pred_train)
rmse_in = np.sqrt(mean_squared_error(y_train_split, y_pred_train))
# 计算样本内RMAE（MAE除以目标变量均值）
y_train_mean = np.mean(y_train_split)
rmae_in = mae_in / y_train_mean if y_train_mean != 0 else np.nan  # 避免除零

# ---------------- 3. 样本外性能（测试集） ----------------
X_test_const = sm.add_constant(X_test_split)
y_pred_log = model.predict(X_test_const)
mae_out = mean_absolute_error(y_test_split, y_pred_log)
rmse_out = np.sqrt(mean_squared_error(y_test_split, y_pred_log))
# 计算样本外RMAE
y_test_mean = np.mean(y_test_split)
rmae_out = mae_out / y_test_mean if y_test_mean != 0 else np.nan


In [83]:
# 4. 6折交叉验证（基于训练集数据）
kf = KFold(n_splits=6, shuffle=True, random_state=42)

cv_mae_scores = []
cv_rmse_scores = []
cv_rmae_scores = []

for train_idx, val_idx in kf.split(X_train_split):
    # 划分训练集和验证集
    X_fold_train, X_fold_val = X_train_split.iloc[train_idx], X_train_split.iloc[val_idx]
    y_fold_train, y_fold_val = y_train_split.iloc[train_idx], y_train_split.iloc[val_idx]
    
    # 添加常数项并训练模型
    X_fold_train_const = sm.add_constant(X_fold_train)
    X_fold_val_const = sm.add_constant(X_fold_val)
    
    fold_model = sm.OLS(y_fold_train, X_fold_train_const).fit()
    y_fold_pred = fold_model.predict(X_fold_val_const)
    
    # 计算各种指标
    mae = mean_absolute_error(y_fold_val, y_fold_pred)
    rmse = np.sqrt(mean_squared_error(y_fold_val, y_fold_pred))
    rmae = mae / np.mean(y_fold_val)
    
    cv_mae_scores.append(mae)
    cv_rmse_scores.append(rmse)
    cv_rmae_scores.append(rmae)

# 5. 打印结果
print("=== OLS模型性能 ===")
print(f"样本内：MAE={mae_in:.4f}, RMAE={rmae_in:.4f}")
print(f"样本外：MAE={mae_out:.4f}, RMAE={rmae_out:.4f}")
print(f"6折交叉验证：")
print(f"  MAE: {np.mean(cv_mae_scores):.4f} (±{np.std(cv_mae_scores):.4f})")
print(f"  RMSE: {np.mean(cv_rmse_scores):.4f} (±{np.std(cv_rmse_scores):.4f})")
print(f"  RMAE: {np.mean(cv_rmae_scores):.4f} (±{np.std(cv_rmae_scores):.4f})")

=== OLS模型性能 ===
样本内：MAE=0.1711, RMAE=0.0120
样本外：MAE=0.1721, RMAE=0.0121
6折交叉验证：
  MAE: 0.1739 (±0.0015)
  RMSE: 0.2435 (±0.0025)
  RMAE: 0.0122 (±0.0001)


# 岭回归

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

ridge_model = Ridge(
    alpha=0.01,          # 正则化强度（>0启用L2正则化，值越大约束越强）
    fit_intercept=False, # 因X含常数项，无需额外拟合截距（避免重复）
    random_state=111     # 固定随机种子，确保模型可复现
)

# 用划分后的训练集拟合模型
ridge_model.fit(X_train_split, y_train_split)

Ridge(alpha=0.01, fit_intercept=False, random_state=111)

In [ ]:
# ---------------- 2. 预测与性能评估（转换为原始房价水平） ----------------
# 核心：因变量是log(Price)，需通过np.exp()反转换为原始房价后计算误差

# 2.1 样本内性能（训练集：X_train_split）
# 预测log值并转换为原始房价
y_train_pred_log = ridge_model.predict(X_train_split)
y_train_pred = np.exp(y_train_pred_log)  # 预测原始房价
y_train_actual = np.exp(y_train_split)   # 实际原始房价（训练集）

# 计算样本内MAE和RMAE
mae_train = mean_absolute_error(y_train_actual, y_train_pred)
y_train_mean = y_train_actual.mean()     # 训练集实际房价的平均值
rmae_train = mae_train / y_train_mean    # RMAE = MAE / 实际均值（相对误差）

# 2.2 样本外性能（测试集：X_test_split）
y_test_pred_log = ridge_model.predict(X_test_split)
y_test_pred = np.exp(y_test_pred_log)    # 预测原始房价
y_test_actual = np.exp(y_test_split)     # 实际原始房价（测试集）

# 计算样本外MAE和RMAE
mae_test = mean_absolute_error(y_test_actual, y_test_pred)
y_test_mean = y_test_actual.mean()       # 测试集实际房价的平均值
rmae_test = mae_test / y_test_mean       # 相对误差



In [ ]:
# ---------------- 3. 6折交叉验证性能（全量数据评估稳定性） ----------------
# 自定义评分函数：基于原始房价计算MAE和RMAE
def score_mae_raw(model, X, y):
    """自定义MAE评分（原始房价）"""
    y_pred_log = model.predict(X)
    y_pred_raw = np.exp(y_pred_log)
    y_actual_raw = np.exp(y)
    return mean_absolute_error(y_actual_raw, y_pred_raw)

def score_rmae_raw(model, X, y):
    """自定义RMAE评分（原始房价：MAE / 实际均值）"""
    y_pred_log = model.predict(X)
    y_pred_raw = np.exp(y_pred_log)
    y_actual_raw = np.exp(y)
    mae = mean_absolute_error(y_actual_raw, y_pred_raw)
    y_mean = y_actual_raw.mean()  # 该折数据的实际房价均值
    return mae / y_mean

# 用全量数据（X_train, y）进行6折交叉验证
cv_mae = cross_val_score(
    ridge_model, X_train, y,
    cv=6,
    scoring=score_mae_raw
).mean()  # 6折MAE的平均值

cv_rmae = cross_val_score(
    ridge_model, X_train, y,
    cv=6,
    scoring=score_rmae_raw
).mean()  # 6折RMAE的平均值

# ---------------- 4. 打印性能报告 ----------------
print("="*70)
print("岭回归（Ridge Regression）模型性能报告（原始住房价格水平）")
print("="*70)
print(f"数据规模：训练集={len(X_train_split)} | 测试集={len(X_test_split)} | 全量数据={len(X_train)}")
print(f"正则化强度（alpha）：{ridge_model.alpha}")
print("-"*70)

print("\n【1. 样本内性能（训练集）】")
print(f"MAE（平均绝对误差）：{mae_train:.4f}")  # 单位：原始房价单位（如元）
print(f"RMAE（相对平均绝对误差）：{rmae_train:.4f}（误差占训练集实际均值的比例）")

print("\n【2. 样本外性能（测试集）】")
print(f"MAE（平均绝对误差）：{mae_test:.4f}")
print(f"RMAE（相对平均绝对误差）：{rmae_test:.4f}（误差占测试集实际均值的比例）")

print("\n【3. 6折交叉验证性能（全量数据）】")
print(f"平均MAE：{cv_mae:.4f}")
print(f"平均RMAE：{cv_rmae:.4f}（误差占全量数据实际均值的平均比例）")
print("\n" + "="*70)

岭回归（Ridge Regression）模型性能报告（原始住房价格水平）
数据规模：训练集=83096 | 测试集=20775 | 全量数据=103871
正则化强度（alpha）：0.01
----------------------------------------------------------------------

【1. 样本内性能（训练集）】
MAE（平均绝对误差）：511692.1551
RMAE（相对平均绝对误差）：0.2255（误差占训练集实际均值的比例）

【2. 样本外性能（测试集）】
MAE（平均绝对误差）：496882.1580
RMAE（相对平均绝对误差）：0.2224（误差占测试集实际均值的比例）

【3. 6折交叉验证性能（全量数据）】
平均MAE：2522027.2875
平均RMAE：0.8425（误差占全量数据实际均值的平均比例）



# 弹性网络

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# ---------------- 1. 训练弹性网络模型（含L1+L2正则化） ----------------
elastic_net = ElasticNet(
    alpha=0.01,          # 正则化总强度（>0启用正则化，值越大约束越强）
    l1_ratio=0.5,        # L1正则化比例（0=纯岭回归，1=纯Lasso，0.5=平衡两者）
    fit_intercept=False, # X含常数项，无需额外拟合截距（避免重复）
    random_state=111     # 固定随机种子，确保模型可复现
)

# 用划分后的训练集拟合模型（仅训练集参与拟合，保证泛化能力评估客观）
elastic_net.fit(X_train_split, y_train_split)



c:\Users\heart\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.358e+03, tolerance: 1.697e+03
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0.01, fit_intercept=False, random_state=111)

In [ ]:
# ---------------- 2. 预测与性能评估（转换为原始房价水平） ----------------
# 核心逻辑：因变量是log(Price)，需通过np.exp()反转换为原始房价，再计算误差

# 2.1 样本内性能（训练集：评估拟合能力）
# 预测log值并转换为原始房价
y_train_pred_log = elastic_net.predict(X_train_split)
y_train_pred = np.exp(y_train_pred_log)  # 训练集预测原始房价
y_train_actual = np.exp(y_train_split)   # 训练集实际原始房价

# 计算样本内MAE和RMAE
mae_train = mean_absolute_error(y_train_actual, y_train_pred)
y_train_mean = y_train_actual.mean()     # 训练集实际房价均值
rmae_train = mae_train / y_train_mean    # RMAE=MAE/实际均值（相对误差，无量纲）

# 2.2 样本外性能（测试集：评估泛化能力，最关键指标）
y_test_pred_log = elastic_net.predict(X_test_split)
y_test_pred = np.exp(y_test_pred_log)    # 测试集预测原始房价
y_test_actual = np.exp(y_test_split)     # 测试集实际原始房价

# 计算样本外MAE和RMAE
mae_test = mean_absolute_error(y_test_actual, y_test_pred)
y_test_mean = y_test_actual.mean()       # 测试集实际房价均值
rmae_test = mae_test / y_test_mean       # 相对误差



In [ ]:
# ---------------- 3. 6折交叉验证性能（全量数据：评估稳定性） ----------------
# 自定义评分函数：适配cross_val_score，基于原始房价计算MAE/RMAE
def score_mae_raw(model, X, y):
    """自定义MAE评分（原始房价）"""
    y_pred_log = model.predict(X)
    y_pred_raw = np.exp(y_pred_log)
    y_actual_raw = np.exp(y)
    return mean_absolute_error(y_actual_raw, y_pred_raw)

def score_rmae_raw(model, X, y):
    """自定义RMAE评分（原始房价：MAE/实际均值）"""
    y_pred_log = model.predict(X)
    y_pred_raw = np.exp(y_pred_log)
    y_actual_raw = np.exp(y)
    mae = mean_absolute_error(y_actual_raw, y_pred_raw)
    y_mean = y_actual_raw.mean()  # 单折数据的实际房价均值
    return mae / y_mean

# 用全量数据（X_train, y）执行6折交叉验证
cv_mae = cross_val_score(
    elastic_net, X_train, y,
    cv=6,
    scoring=score_mae_raw
).mean()  # 6折MAE平均值

cv_rmae = cross_val_score(
    elastic_net, X_train, y,
    cv=6,
    scoring=score_rmae_raw
).mean()  # 6折RMAE平均值



c:\Users\heart\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.685e+03, tolerance: 1.727e+03
  model = cd_fast.enet_coordinate_descent(
c:\Users\heart\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.653e+03, tolerance: 1.791e+03
  model = cd_fast.enet_coordinate_descent(
c:\Users\heart\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.298e+03, toleranc

In [ ]:
# ---------------- 4. 打印完整性能报告 ----------------
print("="*75)
print("弹性网络（Elastic Net）模型性能报告（原始住房价格水平）")
print("="*75)
print(f"数据规模：训练集={len(X_train_split)} | 测试集={len(X_test_split)} | 全量数据={len(X_train)}")
print(f"正则化参数：alpha={elastic_net.alpha}，L1比例(l1_ratio)={elastic_net.l1_ratio}")
print("-"*75)

print("\n【1. 样本内性能（训练集）】")
print(f"MAE（平均绝对误差）：{mae_train:.4f}")  # 单位：原始房价单位（如元）
print(f"RMAE（相对平均绝对误差）：{rmae_train:.4f}（误差占训练集实际均值的比例）")

print("\n【2. 样本外性能（测试集）】")
print(f"MAE（平均绝对误差）：{mae_test:.4f}")
print(f"RMAE（相对平均绝对误差）：{rmae_test:.4f}（误差占测试集实际均值的比例）")

print("\n【3. 6折交叉验证性能（全量数据）】")
print(f"平均MAE：{cv_mae:.4f}")
print(f"平均RMAE：{cv_rmae:.4f}（误差占全量数据实际均值的平均比例）")
print("\n" + "="*75)

弹性网络（Elastic Net）模型性能报告（原始住房价格水平）
数据规模：训练集=83096 | 测试集=20775 | 全量数据=103871
正则化参数：alpha=0.01，L1比例(l1_ratio)=0.5
---------------------------------------------------------------------------

【1. 样本内性能（训练集）】
MAE（平均绝对误差）：892876.2180
RMAE（相对平均绝对误差）：0.3934（误差占训练集实际均值的比例）

【2. 样本外性能（测试集）】
MAE（平均绝对误差）：859632.2676
RMAE（相对平均绝对误差）：0.3848（误差占测试集实际均值的比例）

【3. 6折交叉验证性能（全量数据）】
平均MAE：1339976.4393
平均RMAE：0.5328（误差占全量数据实际均值的平均比例）

